<a href="https://colab.research.google.com/github/kmeng01/rome/blob/main/notebooks/causal_trace.ipynb"><img src="https://colab.research.google.com/assets/colab-badge.svg" align="left"/></a>&nbsp;or in a local notebook.

In [1]:
IS_COLAB = False

## Causal Tracing

A demonstration of the double-intervention causal tracing method.

The strategy used by causal tracing is to understand important
states within a transfomer by doing two interventions simultaneously:

1. Corrupt a subset of the input.  In our paper, we corrupt the subject tokens
   to frustrate the ability of the transformer to accurately complete factual
   prompts about the subject.
2. Restore a subset of the internal hidden states.  In our paper, we scan
   hidden states at all layers and all tokens, searching for individual states
   that carry the necessary information for the transformer to recover its
   capability to complete the factual prompt.

The traces of decisive states can be shown on a heatmap.  This notebook
demonstrates the code for conducting causal traces and creating these heatmaps.

In [1]:
%load_ext autoreload
%autoreload 2

The `experiments.causal_trace` module contains a set of functions for running causal traces.

In this notebook, we reproduce, demonstrate and discuss the interesting functions.

We begin by importing several utility functions that deal with tokens and transformer models.

In [2]:
import os, sys, re, json
import string
import torch
import numpy as np
import copy
from collections import defaultdict, Counter
from util import nethook
from util.globals import DATA_DIR
from experiments.causal_trace import (
    ModelAndTokenizer,
    layername,
    guess_subject,
    plot_trace_heatmap,
)
from experiments.causal_trace import (
    make_inputs,
    decode_tokens,
    find_token_range,
    predict_token,
    predict_from_input,
    collect_embedding_std,
)
from dsets import KnownsDataset

In [3]:
torch.set_grad_enabled(False)

In [4]:
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

## USKG

In [5]:
from transformers import (
    HfArgumentParser,
    set_seed,
    AutoTokenizer
)

# from uskg.models.unified.prefixtuning import Model
from uskg.models.unified import finetune, prefixtuning
from uskg.utils.configue import Configure
from uskg.utils.training_arguments import WrappedSeq2SeqTrainingArguments
from uskg.seq2seq_construction import spider as s2s_spider
from uskg.third_party.spider.preprocess.get_tables import dump_db_json_schema
from uskg.third_party.spider import evaluation as sp_eval
from tqdm.notebook import tqdm

# from nltk.stem.wordnet import WordNetLemmatizer
# import stanza

import matplotlib.pyplot as plt
import sqlite3

from experiments import causal_trace_uskg as ctu

In [10]:
mt_uskg = ctu.ModelAndTokenizer_USKG('t5-large-prefix')

Using tokenizer_uskg: hkunlp/from_all_T5_large_prefix_spider_with_cell_value2
Using tokenizer_fast: t5-large
prefix-tuning sequence length is 10.


In [11]:
list(mt_uskg.task_args.seq2seq)

[('constructor', 'seq2seq_construction.spider'),
 ('schema_serialization_with_db_content', True),
 ('target_with_db_id', False)]

In [12]:
mt_uskg.model.pretrain_model.encoder.embed_tokens is mt_uskg.model.pretrain_model.shared, \
mt_uskg.model.pretrain_model.decoder.embed_tokens is mt_uskg.model.pretrain_model.shared

(True, False)

In [13]:
mt_uskg.model.preseqlen

10

In [14]:
# [k for k,v in mt_uskg.model.named_parameters()]
# [k for k,v in mt_uskg.model.named_modules()]

In [15]:
inp = ctu.make_inputs_t5(
    mt_uskg.tokenizer,
    enc_sentences=["Translate to German: My name is Wolfgang and I live in Berlin"],
    dec_prompts=["Mein Name ist Wolfgang"],
    device="cuda:0"
)

In [16]:
out = ctu.run_model_forward_uskg(mt_uskg.model, **inp)

In [17]:
out.keys(), out['logits'].size()

(odict_keys(['logits', 'past_key_values', 'encoder_last_hidden_state']),
 torch.Size([1, 5, 32102]))

In [18]:
logits = out["logits"][0, -1].detach().cpu().numpy()
logits.shape

(32102,)

In [19]:
top_5 = sorted(list(enumerate(logits)), key=lambda p: -p[1])[:5]
top_5

[(11, -1.8642352),
 (6, -9.727753),
 (5, -10.966707),
 (27, -11.037394),
 (213, -12.864212)]

In [20]:
[mt_uskg.tokenizer.decode([p[0]]) for p in top_5]

['and', ',', '.', 'I', 'where']

### Load spider dataset

In [6]:
spider_train_path = '/home/yshao/Projects/SDR-analysis/data/spider/train+ratsql_graph.json'
spider_dev_path = '/home/yshao/Projects/SDR-analysis/data/spider/dev+ratsql_graph.json'
spider_db_dir = '/home/yshao/Projects/language/language/xsp/data/spider/database'

In [7]:
raw_spider_dev = ctu.load_raw_dataset(
    data_filepath = spider_dev_path,
    db_path=spider_db_dir,
#     schema_cache=SCHEMA_CACHE
)
len(raw_spider_dev)

1034

In [8]:
raw_spider_dev[0].keys()

dict_keys(['query', 'question', 'db_id', 'db_path', 'db_table_names', 'db_column_names', 'db_column_types', 'db_primary_keys', 'db_foreign_keys', 'rat_sql_graph'])

In [21]:
mt_uskg.task_args.dataset.use_cache

True

In [22]:
processed_spider_dev = s2s_spider.DevDataset(
    args=mt_uskg.task_args,
    raw_datasets=raw_spider_dev,
    cache_root='../cache')

In [23]:
_id = 130
processed_spider_dev[_id]['text_in'], \
processed_spider_dev[_id]['struct_in'], \
processed_spider_dev[_id]['seq_out']

('What are the names of all European countries with at least 3 manufacturers?',
 '| car_1 | continents : contid , continent ( europe ) | countries : countryid , countryname , continent | car_makers : id , maker , fullname , country | model_list : modelid , maker , model | car_names : makeid , model , make | cars_data : id , mpg , cylinders , edispl , horsepower , weight , accelerate , year',
 "select t1.countryname from countries as t1 join continents as t2 on t1.continent = t2.contid join car_makers as t3 on t1.countryid = t3.country where t2.continent = 'europe' group by t1.countryname having count(*) >= 3;")

In [24]:
_enc_sentence = f"{processed_spider_dev[_id]['text_in']}; structed knowledge: {processed_spider_dev[_id]['struct_in']}"
_toks = mt_uskg.tokenizer.tokenize(_enc_sentence)
len(_toks)

142

In [ ]:
# # _occ_punct = set()

# for _id in range(len(processed_spider_dev)):
#     ex = processed_spider_dev[_id]
# #     _occ_punct.update(set(string.punctuation) & set(ex['seq_out']))
#     if '_(' in ex['struct_in']:
#         print(_id, ex['question'])
#         print(ex['struct_in'])
#         print(ex['seq_out'])
#         print()

In [ ]:
# ## Train set

# raw_spider_train = ctu.load_raw_dataset(
#     data_filepath = spider_train_path,
#     db_path=spider_db_dir,
# )
# processed_spider_train = s2s_spider.TrainDataset(
#     args=mt_uskg.task_args,
#     raw_datasets=raw_spider_train,
#     cache_root='../cache')
# len(processed_spider_train)

In [ ]:
# processed_spider_train[5441]

#### Analysis sample 1 (ID = 0)

In [32]:
ex = processed_spider_dev[0]
ex['question'], ex['seq_out']

('How many singers do we have?', 'select count(*) from singer')

In [33]:
text_in = ex['text_in']
struct_in = ex['struct_in']

enc_sentence = f"{text_in}; structed knowledge: {struct_in}"
dec_prompt = 'select count(*) from'

inp = ctu.make_inputs_t5(
    mt_uskg.tokenizer,
    enc_sentences=[enc_sentence],
    dec_prompts=[dec_prompt],
    device="cuda:0"
)

In [34]:
mt_uskg.tokenizer.decode(inp['input_ids'][0]), mt_uskg.tokenizer.decode(inp['decoder_input_ids'][0])

('How many singers do we have?; structed knowledge: | concert_singer | stadium : stadium_id, location, name, capacity, highest, lowest, average | singer : singer_id, name, country, song_name, song_release_year, age, is_male | concert : concert_id, concert_name, theme, stadium_id, year | singer_in_concert : concert_id, singer_id</s>',
 '<pad> select count(*) from')

In [35]:
out = ctu.run_model_forward_uskg(mt_uskg.model, **inp)

In [36]:
out.keys(), out['logits'].size()

(odict_keys(['logits', 'past_key_values', 'encoder_last_hidden_state']),
 torch.Size([1, 7, 32102]))

In [37]:
logits = out["logits"][0, -1].detach().cpu().numpy()
logits.shape

(32102,)

In [38]:
top_5 = sorted(list(enumerate(logits)), key=lambda p: -p[1])[:5]
top_5

[(7634, -5.629301),
 (6721, -15.1248665),
 (10159, -17.77869),
 (2377, -18.263933),
 (8782, -18.631098)]

In [39]:
[mt_uskg.tokenizer.decode([p[0]]) for p in top_5]

['singer', 'vocal', 'sing', 'artist', 'singing']

In [40]:
# e_range = ctu.find_token_range(mt_uskg.tokenizer, inp["input_ids"][0], 'singer')
# e_range

In [41]:
text_range, struct_range = ctu.find_text_struct_in_range(mt_uskg.tokenizer, inp["input_ids"][0])
text_range, struct_range

((0, 8), (15, 125))

In [42]:
tb, te = text_range
sb, se = struct_range
mt_uskg.tokenizer.decode(inp['input_ids'][0]), \
mt_uskg.tokenizer.decode(inp['input_ids'][0][tb:te]), \
mt_uskg.tokenizer.decode(inp['input_ids'][0][sb:se])

('How many singers do we have?; structed knowledge: | concert_singer | stadium : stadium_id, location, name, capacity, highest, lowest, average | singer : singer_id, name, country, song_name, song_release_year, age, is_male | concert : concert_id, concert_name, theme, stadium_id, year | singer_in_concert : concert_id, singer_id</s>',
 'How many singers do we have?',
 '| concert_singer | stadium : stadium_id, location, name, capacity, highest, lowest, average | singer : singer_id, name, country, song_name, song_release_year, age, is_male | concert : concert_id, concert_name, theme, stadium_id, year | singer_in_concert : concert_id, singer_id')

In [43]:
result = ctu.calculate_hidden_flow_uskg(
    mt_uskg,
    enc_sentence=enc_sentence,
    dec_prompt=dec_prompt,
    subject='singer',
    replace=True,
)

calculate_hidden_flow_uskg(): corrupted input: *How *many *singer *s *do *we *have *? *; * *struct *e *d *knowledge *: *| *concert *_ *s *inger *| *stadium * *: *stadium *_ *i *d * *, *location * *, *name * *, *capacity * *, *highest * *, *lowest * *, *average *| *singer * *: *singer *_ *i *d * *, *name * *, *country * *, *song *_ *name * *, *song *_ *release *_ *year * *, *age * *, *is *_ *male *| *concert * *: *concert *_ *i *d * *, *concert *_ *name * *, *theme * *, *stadium *_ *i *d * *, *year *| *singer *_ *in *_ *conce *r *t * *: *concert *_ *i *d * *, *singer *_ *i *d *</s>


trace_important_states_uskg.encoder:   0%|          | 0/1512 [00:00<?, ?it/s]

trace_important_states_uskg.decoder:   0%|          | 0/84 [00:00<?, ?it/s]

In [44]:
result.keys(), result['correct_prediction']

(dict_keys(['scores', 'low_score', 'high_score', 'input_ids', 'input_tokens', 'dec_input_ids', 'dec_input_tokens', 'subject_range', 'answer', 'window', 'correct_prediction', 'kind']),
 True)

In [45]:
result['low_score']

3.2487089356436627e-06

In [ ]:
ctu.plot_trace_heatmap_t5(result)

### Helpers
- merged in create_analysis_sample_dicts()

#### Evaluator

In [867]:
table_path = '/home/yshao/Projects/language/language/xsp/data/spider/tables.json'
db_dir = '/home/yshao/Projects/language/language/xsp/data/spider/database'

In [868]:
kmaps = sp_eval.build_foreign_key_map_from_json(table_path)
evaluator = sp_eval.Evaluator(db_dir=db_dir, kmaps=kmaps, etype='all')

In [869]:
ctu.evaluate_hardness.evaluator = evaluator

In [870]:
# test
_sql_str = 'select t1.birth_date from people as t1 join poker_player as t2 on t1.people_id = t2.people_id order by t2.earnings asc limit 1'
db_name = 'poker_player'
schema = evaluator.schemas[db_name]
_sql = sp_eval.get_sql(schema, _sql_str)
sp_eval.count_component1(_sql), sp_eval.count_component2(_sql), sp_eval.count_others(_sql), \
evaluator.eval_hardness(_sql)

(3, 0, 0, 'hard')

#### Hardness

In [58]:
ctu.evaluate_hardness(_sql_str, db_name, evaluator=evaluator)

'hard'

In [59]:
ctu.evaluate_hardness.evaluator

#### Node role

In [29]:
dec_prompt = 'select avg(age), min(age), max(age) from'
ctu.detect_node_role(dec_prompt)

'from'

#### Text match

In [73]:
a_dicts = ctu.create_analysis_sample_dicts(
    mt=mt_uskg,
    ex=processed_spider_dev[100],
    subject_type='table'
)
len(a_dicts), [d['expect'] for d in a_dicts]

(4, ['car_makers', 'cars_data', 'car_names', 'model_list'])

In [75]:
a_ex = a_dicts[2]
ctu.check_table_text_match(a_ex, 'car_names')

'partial'

In [76]:
a_ex['text_in']

'What is the name of the different car makers who produced a car in 1970?'

### Exp0: Study the influence of corrupting a token

In [75]:
_id = 5
ex = processed_spider_dev[_id]
ex['question'], ex['seq_out']

('What is the average, minimum, and maximum age for all French singers?',
 "select avg(age), min(age), max(age) from singer where country = 'France'")

In [76]:
text_in = ex['text_in']
struct_in = ex['struct_in']

enc_sentence = f"{text_in}; structed knowledge: {struct_in}"

# expect = 'singer'
# expect = 'age'
expect = 'country'
dec_prompt = ctu.make_dec_prompt(ex['seq_out'], expect)

ex['enc_sentence'] = enc_sentence
ex['dec_prompt'] = dec_prompt
ex['expect'] = expect
dec_prompt, expect

('select avg(age), min(age), max(age) from singer where', 'country')

In [77]:
ans_toks = decode_tokens(mt_uskg.tokenizer, mt_uskg.tokenizer.encode(expect, add_special_tokens=False))
ans_toks

['country']

In [78]:
inp = ctu.make_inputs_t5(
    mt_uskg.tokenizer,
    [enc_sentence],
    [dec_prompt],
    answer=expect,
    device='cuda'
)

answer_len = len(mt_uskg.tokenizer.tokenize(expect))
with torch.no_grad():
    answers_t, base_score = [d[0] for d in ctu.predict_from_input_uskg_multi_token(mt_uskg.model, inp, pred_len=answer_len)]
base_score = base_score.min().item()
answer = ctu.decode_sentences(mt_uskg.tokenizer, answers_t)
base_score, answer

(0.9999990463256836, 'country')

In [79]:
answers_t

tensor([684], device='cuda:0')

In [80]:
result_d = ctu.token_corruption_influence_uskg(
    mt_uskg,
#     enc_sentence=enc_sentence,
#     dec_prompt=dec_prompt,
#     expect=expect,
    ex,
    replace=True,
    use_tqdm=True,
    skips=('token',)
)

Corrupt effect: columns:   0%|          | 0/17 [00:00<?, ?it/s]

Corrupt effect: tables:   0%|          | 0/4 [00:00<?, ?it/s]

In [81]:
result_d.keys()

dict_keys(['enc_sentence', 'dec_prompt', 'expect', 'base_score', 'answers_t', 'answer', 'res_list'])

In [ ]:
result = result_d['res_list']
result

In [ ]:
[d for d in result if d['corrpt_type'] != 'token']

In [ ]:
# ID = 503

_seq_len = len([d for d in result if d['corrpt_type'] == 'token'])
_tags = [[False, False, False] for _ in range(_seq_len)]  # (is_span_start, is_span_end, is_unit)
for i, d in enumerate(result):
    if d['corrpt_drop'] > 0.5:
        s, e = d['corrpt_idx']
        if e - s > 1:
            _tags[s][0] = True
            _tags[e-1][1] = True
        else:
            _tags[s][2] = True

l = []
for i in range(_seq_len):
    id_str = str(i)
    if _tags[i][2]:
        id_str = f'*{i}*'
    if _tags[i][0]:
        id_str = 'S' + id_str
    if _tags[i][1]:
        id_str = id_str + 'E'
    
    d = result[i]
    l.append(f'{id_str}\t{d["corrpt_token"]}\t{d["corrpt_drop"]}')
#         l.append(f'{i}\t{d["corrpt_token"]}\t{d["corrpt_drop"]}')
print('\n'.join(l))

#### Exp3 (moved to py script)

In [25]:
expect_type = 'column'

res_dir = '/home/yshao/Projects/rome/results/exp3_relational_nodes_mutual'
os.makedirs(res_dir, exist_ok=True)
res_path = os.path.join(res_dir, f'exp=3.0_dev_{expect_type}.jsonl')

#### Load results

In [26]:
with open(res_path, 'r') as f:
    all_results = [json.loads(l) for l in f]
len(all_results)

2014

In [27]:
all_results[0].keys(), all_results[0]['res_list'][0].keys()

(dict_keys(['enc_sentence', 'seq_out', 'dec_prompt', 'expect', 'expect_type', 'db_id', 'expect_input_ranges', 'expect_table', 'answer', 'base_score', 'answers_t', 'correct_prediction', 'category', 'res_list', 'ex_id']),
 dict_keys(['corrpt_type', 'corrpt_idx', 'corrpt_token', 'corrpt_score', 'corrpt_drop']))

In [28]:
# # One-time patches
# # patch 1, for those d without "correct_prediction", set it True 
# # patch 2, for each d, add "expect_table" which is table of "expect" column 
# # patch 3, for each d, add ex_id 
# # patch 4, remove d where db_id = 'orchestra' and expect = 'orchestra' (it is duplicated in struct, should be skipped for now)

# in_path = os.path.join(res_dir, 'exp=3.0_dev_column_old.jsonl')
# out_path = os.path.join(res_dir, 'exp=3.0_dev_column.jsonl')


# with open(in_path, 'r') as f:
#     all_results = [json.loads(l) for l in f]

# keep_results = []
# for d in all_results:
#     if (d['db_id'] == 'orchestra') and (d['expect'] == 'orchestra'):
#         continue
#     keep_results.append(d)

# # ex_id = 0
# # for d in all_results:
# #     d_text_in = d['enc_sentence'].split(ctu.USKG_SPLITTER)[0]
# #     while True:
# #         spider_ex = processed_spider_dev[ex_id]
# #         if spider_ex['text_in'] != d_text_in:
# #             ex_id += 1
# #         else:
# #             break
# # #     db_id = spider_ex['db_id']
# # #     col2table = db_col2table_cache[db_id]
# # #     d['expect_table'] = col2table[d['expect']][0]
# #     d['ex_id'] = ex_id

# with open(out_path, 'w') as f:
#     for d in keep_results:
#         f.write(json.dumps(d, indent=None) + '\n')

# # ex_id
# len(keep_results)

In [29]:
db_col2table_cache = dict()   # db_id -> col2table

scores_per_rel = {
    k: [] 
    for k in ['self', 'self_table', 'self_col', 
              'other_col', 'other_table', 'self_col_max', 'other_col_max', 'other_table_max']}
n_corr_pred = 0

for d in all_results:
    if not d['correct_prediction']:
        continue
        
    n_corr_pred += 1
    ex_id = d['ex_id']
    spider_ex = processed_spider_dev[ex_id]
    db_id = spider_ex['db_id']
    
    # TODO: make this a function: get_col2table(struct_in)
    if db_col2table_cache.get(db_id) is None:
        struct_in = spider_ex['struct_in']
        parsed_struct_in = ctu.parse_struct_in(struct_in)
        # parsed_struct_in_cache[db_id] = parsed_struct_in
    
        col2table = defaultdict(list)
        db_id_t, tables = parsed_struct_in
        for table_name_t, cols in tables:
            for col_name_t, vals in cols:
                _, table_name, _ = table_name_t
                _, col_name, _ = col_name_t
                col2table[col_name].append(table_name)
        db_col2table_cache[db_id] = col2table
    col2table = db_col2table_cache[db_id]
    
    expect = d['expect']
    tab = d['expect_table']
    
    corrupt_drop_dict = dict()
    
    corrupt_drop_dict['self'] = None
    corrupt_drop_dict['self_table'] = None
    corrupt_drop_dict['self_col'] = []
    corrupt_drop_dict['other_col'] = []
    corrupt_drop_dict['other_table'] = []
    for res in d['res_list']:
        _drop = res['corrpt_drop']
        _is_other = True
        if res['corrpt_token'] == expect:
            corrupt_drop_dict['self'] = _drop
            _is_other = False
            if (res['corrpt_type'] == 'column') and (d['expect_type'] == 'column'):
                corrupt_drop_dict['self_col'].append(_drop)
        if res['corrpt_token'] == tab:
            corrupt_drop_dict['self_table'] = _drop
            _is_other = False
        if (res['corrpt_type'] == 'column') and (d['expect_type'] == 'table'):
            _c = res['corrpt_token']
            _t = expect
            if _t in col2table[_c]:
                # corrupted is a column of expect table 
                corrupt_drop_dict['self_col'].append(_drop)
                # print(_c, _t)
                is_other = False
        
        if not _is_other:
            continue    
        if res['corrpt_type'] == 'column':
            corrupt_drop_dict['other_col'].append(_drop)
        elif res['corrpt_type'] == 'table':
            corrupt_drop_dict['other_table'].append(_drop)
            
    # corrupt_drop_dict['self_col_max'] = max(corrupt_drop_dict['self_col']) if corrupt_drop_dict['self_col'] else None
    if corrupt_drop_dict['self_col']:
        corrupt_drop_dict['self_col_max'] = max(corrupt_drop_dict['self_col'])
    corrupt_drop_dict['other_col_max'] = max(corrupt_drop_dict['other_col'])
    corrupt_drop_dict['other_table_max'] = max(corrupt_drop_dict['other_table'])
    
#     scores_per_rel['self'].append(corrupt_drop_dict['self'])
#     scores_per_rel['self_table'].append(corrupt_drop_dict['self_table'])
#     scores_per_rel['self_col'].extend(corrupt_drop_dict['self_col'])
#     scores_per_rel['other_col'].extend(corrupt_drop_dict['other_col'])
#     scores_per_rel['other_table'].extend(corrupt_drop_dict['other_table'])
#     scores_per_rel['other_col_max'].append(corrupt_drop_dict['other_col_max'])
#     scores_per_rel['other_table_max'].append(corrupt_drop_dict['other_table_max'])
#     if corrupt_drop_dict['self_col_max'] is not None:
#         scores_per_rel['self_col_max'].append(corrupt_drop_dict['self_col_max'])
    for k, v in corrupt_drop_dict.items():
        scores_per_rel[k].extend(ctu.ensure_list(v))
    
    d['corrupt_drop_dict'] = corrupt_drop_dict
    
[(k, len(scores_per_rel[k])) for k in scores_per_rel]

[('self', 1445),
 ('self_table', 1445),
 ('self_col', 1445),
 ('other_col', 23164),
 ('other_table', 4787),
 ('self_col_max', 1445),
 ('other_col_max', 1445),
 ('other_table_max', 1445)]

In [30]:
{(k, (None in l)) for k, l in scores_per_rel.items()}

{('other_col', False),
 ('other_col_max', False),
 ('other_table', False),
 ('other_table_max', False),
 ('self', False),
 ('self_col', False),
 ('self_col_max', False),
 ('self_table', False)}

In [31]:
[(k, np.mean(scores_per_rel[k])) for k in scores_per_rel]

[('self', 0.6091220908314937),
 ('self_table', 0.06457461528439588),
 ('self_col', 0.6098740620477409),
 ('other_col', 0.0032386777438926126),
 ('other_table', 0.008444325884166957),
 ('self_col_max', 0.6098740620477409),
 ('other_col_max', 0.039504879676591824),
 ('other_table_max', 0.027120517446089714)]

In [ ]:
# self_table_effective_ids = []

# for i, d in enumerate(all_results):
#     self_table_drop = d.get('self_table_drop', 0.0)
#     if self_table_drop > 0.5:
#         self_table_effective_ids.append(i)
# len(self_table_effective_ids)

In [271]:
self_col_effective_ids = []

for i, d in enumerate(all_results):
    self_col_drop = d.get('self_col_max_drop', 0.0)
    if (self_col_drop is not None) and (self_col_drop > 0.5):
        self_col_effective_ids.append(i)
len(self_col_effective_ids)

95

In [273]:
print(self_col_effective_ids)

[135, 184, 190, 191, 194, 196, 200, 203, 204, 206, 207, 209, 224, 226, 235, 246, 247, 268, 273, 275, 278, 280, 281, 282, 285, 287, 294, 298, 316, 395, 402, 423, 446, 459, 463, 467, 471, 489, 491, 495, 531, 638, 642, 711, 719, 720, 750, 763, 764, 781, 794, 795, 842, 843, 848, 977, 978, 1020, 1022, 1025, 1038, 1039, 1040, 1044, 1061, 1065, 1071, 1118, 1302, 1304, 1308, 1310, 1358, 1359, 1360, 1414, 1416, 1418, 1422, 1424, 1440, 1442, 1443, 1456, 1469, 1474, 1486, 1489, 1499, 1641, 1644, 1667, 1671, 1689, 1690]


In [ ]:
# Any type of samples make self-table effective? More with JOIN 

for i in self_col_effective_ids:
    res_d = all_results[i]
    print(i, res_d['dec_prompt'], '-->', res_d['expect'])


In [ ]:
_id = 263
res_d = all_results[_id]
result = res_d['res_list']
# _seq_len = max([d['corrpt_idx'][1] - 1 for d in result])
# _tags = [[False, False, False] for _ in range(_seq_len)]  # (is_span_start, is_span_end, is_unit)
# for i, d in enumerate(result):
#     if d['corrpt_drop'] > 0.5:
#         s, e = d['corrpt_idx']
#         if e - s > 1:
#             _tags[s][0] = True
#             _tags[e-1][1] = True
#         else:
#             _tags[s][2] = True

# spider_ex = processed_spider_dev[res_d['ex_id']]
print(res_d['enc_sentence'])
print(res_d['dec_prompt'], '-->', res_d['expect'])
print()

l = []
for d in result:
    _prefix = ''
    db_id = spider_ex['db_id']
    
    if d["corrpt_type"] == 'column' and d["corrpt_token"] == res_d['expect']:
        _prefix = '**'
    elif d["corrpt_type"] == 'table' and d["corrpt_token"] == res_d['expect_table']:
        _prefix = '*>'
    l.append(f'{_prefix}{d["corrpt_type"]}\t{d["corrpt_token"]}\t{d["corrpt_drop"]}')
#         l.append(f'{i}\t{d["corrpt_token"]}\t{d["corrpt_drop"]}')
print('\n'.join(l))

#### Split by different aspects

In [373]:
scores_per_rel_by_aspect = defaultdict(lambda: defaultdict(lambda: defaultdict(list)))      # [asp_k, asp_v, rel] -> scores 
scores_avg_per_rel_by_aspect = defaultdict(lambda: defaultdict(lambda: defaultdict(float))) # [asp_k, asp_v, rel] -> avg 
scores_cnt_per_rel_by_aspect = defaultdict(lambda: defaultdict(int))  # [asp_k, asp_v] -> scores 

In [374]:
for d in all_results:
    if 'corrupt_drop_dict' not in d:
        continue
    for asp_k, asp_v in d['category'].items():
        scores_cnt_per_rel_by_aspect[asp_k][asp_v] += 1
        for rel_k, rel_score in d['corrupt_drop_dict'].items():
            scores_per_rel_by_aspect[asp_k][asp_v][rel_k].extend(ctu.ensure_list(rel_score))

for asp_k, d1 in scores_per_rel_by_aspect.items():
    for asp_v, d2 in d1.items():
        for rel_k, rel_scores in d2.items():
            avg_score = np.mean(rel_scores)
            scores_avg_per_rel_by_aspect[asp_k][asp_v][rel_k] = avg_score

In [375]:
# asp_k, asp_v, rel_k, rel_scores

In [376]:
scores_cnt_per_rel_by_aspect

defaultdict(<function __main__.<lambda>()>,
            {'sql_hardness': defaultdict(int,
                         {'easy': 252,
                          'medium': 587,
                          'hard': 304,
                          'extra': 413}),
             'node_role': defaultdict(int,
                         {'from': 1096, 'join': 454, 'select': 4, 'where': 2}),
             'text_match': defaultdict(int,
                         {'exact': 1051, 'no-match': 362, 'partial': 143})})

In [377]:
scores_avg_per_rel_by_aspect['overall'] = {k : np.mean(scores_per_rel[k]) for k in scores_per_rel}

In [ ]:
scores_avg_per_rel_by_aspect

In [ ]:
dump_d = ctu.nested_json_processing(scores_avg_per_rel_by_aspect, func=lambda x: np.format_float_positional(x, precision=4, min_digits=4))

for asp_k, d1 in dump_d.items():
    if asp_k == 'overall':
        d1['n_samples'] = len(scores_per_rel['self'])
        continue
    for asp_v, d2 in d1.items():
        d2['n_samples'] = scores_cnt_per_rel_by_aspect[asp_k][asp_v]
        
dump_d

In [380]:
res_dir = '/home/yshao/Projects/rome/results/exp3_relational_nodes_mutual'
dump_path = os.path.join(res_dir, f'summ-exp=3.0_dev_{expect_type}.jsonl')

In [381]:
with open(dump_path, 'w') as f:
    json.dump(dump_d, f, indent=1)

In [275]:
## Hardness

samples_by_hardness = defaultdict(list)

for res_d in all_results:
    if not res_d['correct_prediction']:
        continue
#     hardness = spider_id2hardness[res_d['ex_id']]
    hardness = res_d['category']['sql_hardness']
    samples_by_hardness[hardness].append(res_d)

[(h, len(samples)) for h, samples in samples_by_hardness.items()]

[('easy', 252), ('medium', 587), ('hard', 304), ('extra', 413)]

In [286]:
for h in ['easy', 'medium', 'hard', 'extra']:
    samples = samples_by_hardness[h]
    print(f'Hardness: {h} ({len(samples)})')
    # print('[Example sql]', samples[0]['seq_out'])
    
    for rel in scores_per_rel.keys():
        _rel_k = rel + '_drop'
        rel_scores = [x for res_d in samples for x in ctu.ensure_list(res_d[_rel_k]) if x is not None]
        avg_score = np.mean(rel_scores, )
        print(f'{rel}\t{avg_score:.4f}\t')
    print()

Hardness: easy (252)
self	0.7473	
self_table	0.7473	
self_col	0.0067	
other_col	0.0035	
other_table	0.0007	
self_col_max	0.0349	
other_col_max	0.0460	
other_table_max	0.0028	

Hardness: medium (587)
self	0.7537	
self_table	0.7537	
self_col	0.0147	
other_col	0.0062	
other_table	0.0072	
self_col_max	0.0711	
other_col_max	0.0741	
other_table_max	0.0274	

Hardness: hard (304)
self	0.7050	
self_table	0.7050	
self_col	0.0154	
other_col	0.0084	
other_table	0.0087	
self_col_max	0.0787	
other_col_max	0.0883	
other_table_max	0.0377	

Hardness: extra (413)
self	0.7371	
self_table	0.7371	
self_col	0.0203	
other_col	0.0076	
other_table	0.0035	
self_col_max	0.0814	
other_col_max	0.0832	
other_table_max	0.0290	



In [ ]:
# rel_scores

In [287]:
## Node role

samples_by_node_role = defaultdict(list)

for res_d in all_results:
    if not res_d['correct_prediction']:
        continue
#     col_role_kw = _detect_column_role(res_d['dec_prompt'])
    role_kw = res_d['category']['node_role']
    samples_by_node_role[role_kw].append(res_d)

[(r, len(samples)) for r, samples in samples_by_node_role.items()]

[('from', 1096), ('join', 454), ('select', 4), ('where', 2)]

In [288]:
for r, samples in samples_by_node_role.items():
    if len(samples) < 10:
        continue
    print(f'Column role: {r} ({len(samples)})')
    # print('[Example sql]', samples[0]['seq_out'])
    
    for rel in scores_per_rel.keys():
        _rel_k = rel + '_drop'
        rel_scores = [x for res_d in samples for x in ctu.ensure_list(res_d[_rel_k]) if x is not None]
        avg_score = np.mean(rel_scores)
        print(f'{rel}\t{avg_score:.4f}\t')
    print()

Column role: from (1096)
self	0.7255	
self_table	0.7255	
self_col	0.0109	
other_col	0.0058	
other_table	0.0004	
self_col_max	0.0608	
other_col_max	0.0686	
other_table_max	0.0081	

Column role: join (454)
self	0.7804	
self_table	0.7804	
self_col	0.0288	
other_col	0.0086	
other_table	0.0168	
self_col_max	0.0866	
other_col_max	0.0907	
other_table_max	0.0691	



In [289]:
## Text match 

samples_by_text_match = {k: [] for k in ['exact', 'partial', 'no-match']}

for res_d in all_results:
    if not res_d['correct_prediction']:
        continue
#     spider_ex = processed_spider_dev[res_d['ex_id']]
#     text_match = ctu.check_text_match(spider_ex, col=res_d['expect'], tab=res_d['expect_table'])
    text_match = res_d['category']['text_match']
    samples_by_text_match[text_match].append(res_d)

[(m, len(samples)) for m, samples in samples_by_text_match.items()]

[('exact', 1051), ('partial', 143), ('no-match', 362)]

In [290]:
for m, samples in samples_by_text_match.items():
    print(f'Text-match: {m} ({len(samples)})')
    # print('[Example sql]', samples[0]['seq_out'])
    
    for rel in scores_per_rel.keys():
        _rel_k = rel + '_drop'
        rel_scores = [x for res_d in samples for x in ctu.ensure_list(res_d[_rel_k]) if x is not None]
        avg_score = np.mean(rel_scores)
        print(f'{rel}\t{avg_score:.4f}\t')
    print()

Text-match: exact (1051)
self	0.6815	
self_table	0.6815	
self_col	0.0061	
other_col	0.0038	
other_table	0.0039	
self_col_max	0.0303	
other_col_max	0.0380	
other_table_max	0.0168	

Text-match: partial (143)
self	0.8419	
self_table	0.8419	
self_col	0.0656	
other_col	0.0130	
other_table	0.0028	
self_col_max	0.2214	
other_col_max	0.1938	
other_table_max	0.0644	

Text-match: no-match (362)
self	0.8642	
self_table	0.8642	
self_col	0.0259	
other_col	0.0116	
other_table	0.0116	
self_col_max	0.1361	
other_col_max	0.1344	
other_table_max	0.0370	



### Exp-1: column/table corruption
- In py script

#### Plotting results

In [49]:
exp_name = 'dev_table_alias_encoder-tmp'

res_json_path = f'/home/yshao/Projects/rome/results/exp1_struct_node_restore/{exp_name}.jsonl'
with open(res_json_path, 'r') as f:
    res_dicts = [json.loads(l) for l in f if l]
#     all_str = f.read()
# all_str = all_str.replace('{"ex_id":', '\n{"ex_id":').strip()
# res_dicts = [json.loads(l) for l in all_str.split('\n')]
len(res_dicts)

21

In [50]:
len([r for d in res_dicts for r in d['trace_results']])

55

In [51]:
good_trace_results = []
for d in res_dicts:
    for r in d['trace_results']:
        if r['correct_prediction'] and r['is_good_sample']:
            r['ex_id'] = d['ex_id']
            good_trace_results.append(r)
len(good_trace_results)

3

In [52]:
d.keys(), good_trace_results[0].keys(), type(good_trace_results[0]['scores'])

(dict_keys(['ex_id', 'trace_results']),
 dict_keys(['enc_sentence', 'seq_out', 'dec_prompt', 'expect', 'expect_type', 'db_id', 'expect_input_ranges', 'category', 'low_score', 'high_score', 'input_ids', 'input_tokens', 'dec_input_ids', 'dec_input_tokens', 'subject_range', 'subject_range_individual_indices', 'answer', 'window', 'correct_prediction', 'kind', 'sever_kind', 'scores', 'is_good_sample', 'ex_id']),
 list)

In [53]:
# Generating all plots
fig_save_dir = f'/home/yshao/Projects/rome/results/figs/exp1_struct_node_restore/{exp_name}'

for i, r in enumerate(tqdm(good_trace_results)):
    result = dict(r)

    enc_s, dec_s = result['scores']
    enc_s = np.array(enc_s)
    dec_s = np.array(dec_s)
    result['scores'] = [enc_s, dec_s]

    ex_id = r['ex_id']
    ctu.plot_trace_heatmap_t5(result, savepdf=os.path.join(fig_save_dir, f'{i}-ex_id={ex_id}.pdf'))
    

  0%|          | 0/3 [00:00<?, ?it/s]

findfont: Font family ['Times New Roman'] not found. Falling back to DejaVu Sans.
findfont: Font family ['Times New Roman'] not found. Falling back to DejaVu Sans.
findfont: Font family ['Times New Roman'] not found. Falling back to DejaVu Sans.


#### Finding special samples

In [314]:
for i, r in enumerate(good_trace_results):
    _, dec_s = r['scores']
    ## dec_s: (n_toks, n_layers)
    non_last_token_s = np.array(dec_s)[:-1]
    if (non_last_token_s > 0.5).any():
        print(f'{i}-ex_id={r["ex_id"]}')
    

57-ex_id=69
60-ex_id=70
83-ex_id=139
87-ex_id=143
88-ex_id=144
89-ex_id=145
91-ex_id=153


### Exp-1.1: severing decoder cross-attention
- TODO: merge to py script

#### Single sample (ID = 2)

In [ ]:
_id = 2
ex = processed_spider_dev[_id]
ex['question'], ex['struct_in'], ex['seq_out']

In [ ]:
text_in = ex['text_in']
struct_in = ex['struct_in']

enc_sentence = f"{text_in}; structed knowledge: {struct_in}"
enc_tokenized = mt_uskg.tokenizer(enc_sentence)

token_ranges_dict = ctu.find_struct_name_ranges(mt_uskg.tokenizer, enc_tokenized['input_ids'], struct_in)
token_ranges_dict

In [ ]:
col = 'name'
tok_ranges = token_ranges_dict['col_name_ranges'][col]
tok_indices = [i for s, e in tok_ranges for i in range(s, e)]

dec_prompt = make_dec_prompt(ex['seq_out'], col)

dec_prompt, tok_indices

In [ ]:
result = ctu.calculate_hidden_flow_uskg(
    mt_uskg,
    enc_sentence=enc_sentence,
    dec_prompt=dec_prompt,
    expect=col,
    e_range=tok_indices,
    enc_token_range=[],    # no analysis
    dec_token_range=None,  # full analysis
    tokens_to_mix_individual_indices=True,
    replace=True,
    sever_kind='self_attn'
)

In [ ]:
# no sever
ctu.plot_trace_heatmap_t5(result)
print([f"{s:.2f}" for s in result['scores'][1][-1]])

In [ ]:
# sever cross_attn
ctu.plot_trace_heatmap_t5(result)
print([f"{s:.2f}" for s in result['scores'][1][-1]])

In [ ]:
# sever mlp; a bit better than no severing??
ctu.plot_trace_heatmap_t5(result)
print([f"{s:.2f}" for s in result['scores'][1][-1]])

In [ ]:
# sever self_attn
ctu.plot_trace_heatmap_t5(result)
print([f"{s:.2f}" for s in result['scores'][1][-1]])

In [338]:
len(result['scores'][1][-1])

24

### Exp-2: dirty text recovery
- Corrupt the text, restore different parts of encoder final output
- Idea is to check the existence of contextual understanding (incorporating text info into struct representation)
- (In py script)

#### Single sample
- ID = 2, col = 'name': (name exact match with text)
    - none: wrong; text: correct; struct: correct; col: correct
- ID = 5, col = 'country': (no exact match, but value match with text (French) )
    - none: wrong; text: correct; struct: correct; col: correct
- ID = 145, col = 'year': (no exact match, but value match with text (1980) )
    - none: wrong; text: correct; struct: correct; col: correct
- ID = 7, col = 'song_release_year': (multi-token, and with confusing columns in table (song_name) )
    - none: wrong; text: wrong; struct: correct; col: wrong
    - hypothesis: need clean struct to avoid confusion
- ID = 185, col = 'airportcode': (multi-token, with confusing col (airportname), but have match with text (airport code) )
    - none: wrong; text: correct; struct: wrong (2nd token); col: wrong
    - differs from hypothesis... clean struct should know sql info about the column

In [192]:
_id = 24
ex = processed_spider_dev[_id]
ex['db_id'], ex['question'], ex['struct_in'], ex['seq_out']

('concert_singer',
 'Show the stadium name and capacity with most number of concerts in year 2014 or after.',
 '| concert_singer | stadium : stadium_id , location , name , capacity , highest , lowest , average | singer : singer_id , name , country , song_name , song_release_year , age , is_male | concert : concert_id , concert_name , theme , stadium_id , year | singer_in_concert : concert_id , singer_id',
 'select t2.name, t2.capacity from concert as t1 join stadium as t2 on t1.stadium_id = t2.stadium_id where t1.year >= 2014 group by t2.stadium_id order by count(*) desc limit 1')

In [193]:
text_in = ex['text_in']
struct_in = ex['struct_in']

enc_sentence = f"{text_in}; structed knowledge: {struct_in}"
enc_tokenized = mt_uskg.tokenizer(enc_sentence)

text_range, struct_range = ctu.find_text_struct_in_range(mt_uskg.tokenizer, enc_tokenized['input_ids'])
text_range, struct_range

((0, 17), (24, 134))

In [34]:
token_ranges_dict = ctu.find_struct_name_ranges(mt_uskg.tokenizer, enc_tokenized['input_ids'], struct_in)

col_name_ranges = token_ranges_dict['col_name_ranges']

sql_tokens = ctu.separate_punct(ex['seq_out']).split(' ')

sql_cols = set()
for t in sql_tokens:
    if t in col_name_ranges:
        sql_cols.add(t)
sql_cols

{'name', 'nationality'}

In [35]:
parsed_struct_in = ctu.parse_struct_in(struct_in)
parsed_struct_in

((1, 'orchestra', 'orchestra'),
 [((3, 'conductor', 'conductor'),
   [[(5, 'conductor_id', 'conductor_id'), []],
    [(7, 'name', 'name'), []],
    [(9, 'age', 'age'), []],
    [(11, 'nationality', 'nationality ( USA )'), [(13, 'USA', 'USA')]],
    [(16, 'year_of_work', 'year_of_work'), []]]),
  ((18, 'orchestra', 'orchestra'),
   [[(20, 'orchestra_id', 'orchestra_id'), []],
    [(22, 'orchestra', 'orchestra'), []],
    [(24, 'conductor_id', 'conductor_id'), []],
    [(26, 'record_company', 'record_company'), []],
    [(28, 'year_of_founded', 'year_of_founded'), []],
    [(30, 'major_record_format', 'major_record_format'), []]]),
  ((32, 'performance', 'performance'),
   [[(34, 'performance_id', 'performance_id'), []],
    [(36, 'orchestra_id', 'orchestra_id'), []],
    [(38, 'type', 'type'), []],
    [(40, 'date', 'date'), []],
    [(42, 'official_ratings_(millions)', 'official_ratings_(millions)'), []],
    [(44, 'weekly_rank', 'weekly_rank'), []],
    [(46, 'share', 'share'), []]]),

In [37]:
col = 'nationality'
# tok_ranges = token_ranges_dict['col_name_ranges'][col]
# tok_indices = [i for s, e in tok_ranges for i in range(s, e)]
dec_prompt = ctu.make_dec_prompt(ex['seq_out'], col)
expect = col
dec_prompt, expect

('select name from conductor where', 'nationality')

In [38]:
col_toks = token_ranges_dict['col_name_ranges'][col]
col, col_toks

('nationality', [(45, 51)])

In [39]:
# result = ctu.calculate_hidden_flow_uskg(
#     mt_uskg,
#     enc_sentence=enc_sentence,
#     dec_prompt=dec_prompt,
#     expect=col,
#     e_range=text_range,
#     enc_token_range=[],    # no analysis
#     dec_token_range=None,  # full analysis
#     tokens_to_mix_individual_indices=False,
#     replace=True,
#     sever_kind=None,
# )

inp = ctu.make_inputs_t5(
    mt_uskg.tokenizer,
    [enc_sentence] * 11,
    [dec_prompt] * 11,
    answer=expect)

encoder_text_last_layer_states = [
    (tnum, ctu.layername_uskg(mt_uskg.model, 'encoder', mt_uskg.num_enc_layers - 1))
    for tnum in range(*text_range)
]

encoder_struct_last_layer_states = [
    (tnum, ctu.layername_uskg(mt_uskg.model, 'encoder', mt_uskg.num_enc_layers - 1))
    for tnum in range(*struct_range)
]

encoder_col_last_layer_states = [
    (tnum, ctu.layername_uskg(mt_uskg.model, 'encoder', mt_uskg.num_enc_layers - 1))
    for tnum in col_toks
]

answer_len = len(mt_uskg.tokenizer.tokenize(expect))
answers_t, base_score = [d[0] for d in ctu.predict_from_input_uskg_multi_token(mt_uskg.model, inp, pred_len=answer_len)]

In [40]:
base_score

tensor([1.0000, 1.0000], device='cuda:0')

In [41]:
## Corrupting text: expect wrong pred 

all_ans_probs = ctu.trace_with_repatch_uskg_multi_token(
    model=mt_uskg.model,
    inp=inp,
    states_to_patch=[],
    states_to_unpatch=[],
    answers_t=answers_t,
    tokens_to_mix=text_range,
    tokens_to_mix_individual_indices=False,
    replace=True,
)
all_ans_probs, min(all_ans_probs)

([tensor(0.1734, device='cuda:0'), tensor(1.0000, device='cuda:0')],
 tensor(0.1734, device='cuda:0'))

In [42]:
## Restoring text encoding for decoder, but struct encoding are with dirty text encoding 

all_ans_probs = ctu.trace_with_repatch_uskg_multi_token(
    model=mt_uskg.model,
    inp=inp,
    states_to_patch=encoder_text_last_layer_states,
    states_to_unpatch=[],
    answers_t=answers_t,
    tokens_to_mix=text_range,
    tokens_to_mix_individual_indices=False,
    replace=True,
)
all_ans_probs, min(all_ans_probs)

([tensor(1.0000, device='cuda:0'), tensor(0.9999, device='cuda:0')],
 tensor(0.9999, device='cuda:0'))

In [43]:
## Restoring clean struct encoding but dirty text encoding for decoder
## Prediction being correct means encoder final output has "contextual" or "semantic" understanding of struct_in 

all_ans_probs = ctu.trace_with_repatch_uskg_multi_token(
    model=mt_uskg.model,
    inp=inp,
    states_to_patch=encoder_struct_last_layer_states,
    states_to_unpatch=[],
    answers_t=answers_t,
    tokens_to_mix=text_range,
    tokens_to_mix_individual_indices=False,
    replace=True,
)
all_ans_probs, min(all_ans_probs)

([tensor(1.0000, device='cuda:0'), tensor(1., device='cuda:0')],
 tensor(1.0000, device='cuda:0'))

In [44]:
## Restoring clean col_name encoding but dirty text encoding for decoder (stricter than above)

all_ans_probs = ctu.trace_with_repatch_uskg_multi_token(
    model=mt_uskg.model,
    inp=inp,
    states_to_patch=encoder_col_last_layer_states,
    states_to_unpatch=[],
    answers_t=answers_t,
    tokens_to_mix=text_range,
    tokens_to_mix_individual_indices=False,
    replace=True,
)
all_ans_probs, min(all_ans_probs)

([tensor(1.0000, device='cuda:0'), tensor(1.0000, device='cuda:0')],
 tensor(1.0000, device='cuda:0'))

In [45]:
## For exp-2.1: mutual corruption
# First pass: corrupt text (no restore)
# Second pass: corrupt struct, no restore, reset struct output to first pass

all_ans_probs = ctu.trace_with_repatch_uskg_multi_token(
    model=mt_uskg.model,
    inp=inp,
    states_to_patch=[],
    states_to_unpatch=encoder_struct_last_layer_states,
    answers_t=answers_t,
    tokens_to_mix_1st_pass=text_range,
    tokens_to_mix=struct_range,
    tokens_to_mix_individual_indices=False,
    replace=True,
)
all_ans_probs, min(all_ans_probs)

([tensor(1.0000, device='cuda:0'), tensor(1.0000, device='cuda:0')],
 tensor(1.0000, device='cuda:0'))

#### Load & Check results

In [42]:
res_path = '/home/yshao/Projects/rome/results/exp2_text_struct_interaction/exp=2_dev_table.jsonl'

with open(res_path, 'r') as f:
    all_samples = [json.loads(l) for l in f]
len(all_samples)

1034

In [43]:
# all_samples[5439]['ex_id'], all_samples[5440]['ex_id'], all_samples[-1]['ex_id'], 

In [44]:
total_samples = 0
n_good_samples = 0
n_too_hard = 0      # wrong answer 
n_too_easy = 0      # base - low < 0.5

base_scores = []
low_scores = []

## len = n_good_samples
restore_scores_dict = {
    'text': [],
    'struct': [],
    'node': [],
    'struct_no_node': [],
    'node_corrupt_all': [],
    # 2.0.1: cancelled
    # 'ctname': [],      # col name + table name (col belongs to) 
    # 'catname': [],     # col name + all table names 
    # 'full_table': [],  # full table (col belongs to) 
    # 'all_col': [],     # all col names (regardless of table)
}

## len = total_samples
mutual_scores_dict = {
    f'{text}-{struct}': []
    for text in ['clean_t', 'dc_t', 'dirty_t']
    for struct in ['clean_s', 'dc_s', 'dirty_s']
}


In [45]:
good_samples = []

for i, ex in enumerate(all_samples):
    for d in ex['trace_results']:
        total_samples += 1
        if d['is_good_sample']:
            n_good_samples += 1
            d['ex_id'] = i
            good_samples.append(d)
        elif not d['correct_prediction']:
            n_too_hard += 1
        else:
            assert d['base_score'] - d['low_score'] < 0.5
            n_too_easy += 1
            
total_samples, (n_good_samples, len(good_samples)), n_too_hard, n_too_easy

(1725, (821, 821), 141, 763)

In [36]:
d.keys(), d['mutual_scores'].keys()

(dict_keys(['enc_sentence', 'seq_out', 'dec_prompt', 'expect', 'expect_type', 'db_id', 'expect_input_ranges', 'expect_table', 'answer', 'base_score', 'answers_t', 'correct_prediction', 'category', 'mutual_scores', 'low_score', 'is_good_sample']),
 dict_keys(['clean_t-clean_s', 'clean_t-dc_s', 'clean_t-dirty_s', 'dc_t-clean_s', 'dc_t-dc_s', 'dc_t-dirty_s', 'dirty_t-clean_s', 'dirty_t-dc_s', 'dirty_t-dirty_s']))

In [37]:
for d in good_samples:
    base_scores.append(d['base_score'])
    low_scores.append(d['low_score'])
    for k in restore_scores_dict.keys():
        restore_scores_dict[k].append(d[f'r_{k}_score'])
    for k in mutual_scores_dict.keys():
        mutual_scores_dict[k].append(d['mutual_scores'][k])

In [38]:
len(base_scores), len(restore_scores_dict['text']), len(mutual_scores_dict['dc_t-dc_s'])

(1084, 1084, 1084)

In [39]:
# results for exp2
for k, scores in restore_scores_dict.items():
    avg = np.mean(scores)
    print(f'{k}\t{avg:.4f}')

text	0.6152
struct	0.8505
node	0.7290
struct_no_node	0.1517
node_corrupt_all	0.7480


In [40]:
print(f'Score\tavg_gain\tperc_recover')
for score_label, high_scores in [
    ('base_scores', base_scores),
    ('r_text', restore_scores_dict['text']),
    ('r_struct', restore_scores_dict['struct']),
    ('r_node', restore_scores_dict['node']),
    ('r_struct_no_node', restore_scores_dict['struct_no_node']),
    ('r_node_corrupt_all', restore_scores_dict['node_corrupt_all']),
]:
    avg_gain = np.mean([h - l for h, l in zip(high_scores, low_scores)])
    perc_recover = np.mean([h - l > 0.5 for h, l in zip(high_scores, low_scores)])
    print(f'{score_label}\t{avg_gain:.4f}\t{perc_recover:.4f}')

Score	avg_gain	perc_recover
base_scores	0.9416	1.0000
r_text	0.5663	0.6033
r_struct	0.8016	0.8542
r_node	0.6801	0.7352
r_struct_no_node	0.1028	0.1153
r_node_corrupt_all	0.6990	0.7445


In [41]:
# results for exp2.1.1 
msg = ' '*8
for k_t in ['clean_t', 'dc_t', 'dirty_t']:
    msg += f'{k_t:8s}'
msg += '\n'
for k_s in ['clean_s', 'dc_s', 'dirty_s']:
    msg += f'{k_s:8s}'
    for k_t in ['clean_t', 'dc_t', 'dirty_t']:
        k = f'{k_t}-{k_s}'
        scores = mutual_scores_dict[k]
        avg = np.mean(scores)
        msg += f'{avg:.4f}  '
    msg += '\n'
print(msg)

        clean_t dc_t    dirty_t 
clean_s 0.9905  0.9611  0.8505  
dc_s    0.6152  0.4952  0.0489  
dirty_s 0.3916  0.3469  0.0241  



#### Split by hardness

In [136]:
# exp2 + exp2.1.1 
# res_path = '/home/yshao/Projects/rome/results/exp2_text_struct_interaction/exp=2_dev_column.jsonl'

# with open(res_path, 'r') as f:
#     all_samples = [json.loads(l) for l in f]
# len(all_samples)

In [137]:
# good_samples = []

# for i, ex in enumerate(all_samples):
#     for d in ex['trace_results']:
#         if d['is_good_sample']:
#             d['ex_id'] = i
#             good_samples.append(d)
# len(good_samples)

In [138]:
samples_by_hardness = defaultdict(list)

# for ex in all_samples:
#     for d in ex['trace_results']:
#         if not d['is_good_sample']:
#             continue

for d in good_samples:
#     sql_str = d['seq_out']
#     db_id = d['struct_in'].split('|')[1].strip()    # TODO: add db_id to result during experiment main run 
#     sql = sp_eval.get_sql(evaluator.schemas[db_id], sql_str)
#     hardness = evaluator.eval_hardness(sql)
#     hardness = spider_id2hardness[d['ex_id']]
    hardness = d['category']['sql_hardness']
    samples_by_hardness[hardness].append(d)

In [139]:
[(h, len(samples)) for h, samples in samples_by_hardness.items()]

[('easy', 140), ('medium', 307), ('hard', 168), ('extra', 206)]

In [140]:
for h in ['easy', 'medium', 'hard', 'extra']:
    samples = samples_by_hardness[h]
    print(f'Hardness: {h} ({len(samples)})')
    print('[Example sql]', samples[0]['seq_out'])
    print(f'Score\tavg_gain\tperc_recover')
    
    low_scores = [d['low_score'] for d in samples]
    for score_label in [
        'base_score',
        'r_text_score',
        'r_struct_score',
        'r_node_score',
        'r_struct_no_node_score',
        'r_node_corrupt_all_score'
    ]:
        high_scores = [d[score_label] for d in samples]
        avg_gain = np.mean([h - l for h, l in zip(high_scores, low_scores)])
        perc_recover = np.mean([h - l > 0.5 for h, l in zip(high_scores, low_scores)])
        print(f'{score_label}\t{avg_gain:.4f}\t{perc_recover:.4f}')
    print()

Hardness: easy (140)
[Example sql] select count(*) from singer
Score	avg_gain	perc_recover
base_score	0.8812	1.0000
r_text_score	0.3415	0.3643
r_struct_score	0.8589	0.9714
r_node_score	0.6934	0.7714
r_struct_no_node_score	0.4167	0.4429
r_node_corrupt_all_score	0.8762	1.0000

Hardness: medium (307)
[Example sql] select name, country, age from singer order by age desc
Score	avg_gain	perc_recover
base_score	0.9013	1.0000
r_text_score	0.3883	0.4365
r_struct_score	0.8948	0.9935
r_node_score	0.8010	0.8925
r_struct_no_node_score	0.3597	0.4202
r_node_corrupt_all_score	0.8096	0.8990

Hardness: hard (168)
[Example sql] select song_name from singer where age > (select avg(age) from singer)
Score	avg_gain	perc_recover
base_score	0.8995	1.0000
r_text_score	0.3341	0.3750
r_struct_score	0.8428	0.9286
r_node_score	0.7446	0.8393
r_struct_no_node_score	0.2623	0.2798
r_node_corrupt_all_score	0.7794	0.8690

Hardness: extra (206)
[Example sql] select t2.name, t2.capacity from concert as t1 join stadium as 

#### Split by node role

In [141]:
# role_keyword_pattern = r'\W(select|where|join|group by|having|order by)\W'

In [142]:
# _sql_str = samples_by_hardness['extra'][0]['seq_out']
# _sql_str

In [143]:
# re.findall(role_keyword_pattern, ' ' + _sql_str + ' ')

In [144]:
samples_by_node_role = defaultdict(list)

for d in good_samples:
#     all_kws = re.findall(role_keyword_pattern, ' ' + d['dec_prompt'] + ' ')
#     assert len(all_kws) > 0, d['dec_prompt']
#     col_role_kw = all_kws[-1]
#     col_role_kw = _detect_column_role(d['dec_prompt'])
    role_kw = d['category']['node_role']
    samples_by_node_role[role_kw].append(d)

[(r, len(samples)) for r, samples in samples_by_node_role.items()]

[('from', 533), ('join', 282), ('select', 6)]

In [145]:
for r in samples_by_node_role.keys():
    samples = samples_by_node_role[r]
    if len(samples) < 10:
        continue
    print(f'Column role: {r} ({len(samples)})')
    print('[Example prompt]', samples[0]['dec_prompt'])
    print(f'Score\tavg_gain\tperc_recover')
    
    low_scores = [d['low_score'] for d in samples]
    for score_label in [
        'base_score',
        'r_text_score',
        'r_struct_score',
        'r_node_score',
        'r_struct_no_node_score',
        'r_node_corrupt_all_score'
    ]:
        high_scores = [d[score_label] for d in samples]
        avg_gain = np.mean([h - l for h, l in zip(high_scores, low_scores)])
        perc_recover = np.mean([h - l > 0.5 for h, l in zip(high_scores, low_scores)])
        print(f'{score_label}\t{avg_gain:.4f}\t{perc_recover:.4f}')
    print()

Column role: from (533)
[Example prompt] select count(*) from
Score	avg_gain	perc_recover
base_score	0.8889	1.0000
r_text_score	0.3948	0.4447
r_struct_score	0.8652	0.9681
r_node_score	0.7885	0.8799
r_struct_no_node_score	0.3151	0.3452
r_node_corrupt_all_score	0.8623	0.9644

Column role: join (282)
[Example prompt] select t2.name, t2.capacity from concert as t1 join
Score	avg_gain	perc_recover
base_score	0.8870	1.0000
r_text_score	0.2458	0.2305
r_struct_score	0.8752	0.9752
r_node_score	0.7379	0.8262
r_struct_no_node_score	0.3443	0.4007
r_node_corrupt_all_score	0.6962	0.7801



In [146]:
# res_path = '/home/yshao/Projects/rome/results/exp2_text_struct_interaction/exp=2_dev_column.jsonl'
# with open(res_path, 'r') as f:
#     all_samples = [json.loads(l) for l in f]
# len(all_samples)

#### Split by text match

In [147]:
# d = good_samples[244]
# spider_ex = processed_spider_dev[d['ex_id']]
# col = d['expect']
# tab = d['table']
# d['table'], d['expect'], d['text_in'], ctu.check_text_match(spider_ex, col, tab)

In [148]:
samples_by_text_match = {k: [] for k in ['exact', 'partial', 'no-match']}

for d in good_samples:
#     spider_ex = processed_spider_dev[d['ex_id']]
#     text_match = _check_text_match(spider_ex, d)
    text_match = d['category']['text_match']
    samples_by_text_match[text_match].append(d)

[(m, len(samples)) for m, samples in samples_by_text_match.items()]

[('exact', 551), ('partial', 98), ('no-match', 172)]

In [149]:
for m in ['exact', 'partial', 'no-match']:
    samples = samples_by_text_match[m]
    print(f'Text match: {m} ({len(samples)})')
    print('[Example]', samples[0]['enc_sentence'])
    print(f"[Example] {samples[0]['dec_prompt']} *{samples[0]['expect']}*")
    print(f'Score\tavg_gain\tperc_recover')
    
    low_scores = [d['low_score'] for d in samples]
    for score_label in [
        'base_score',
        'r_text_score',
        'r_struct_score',
        'r_node_score',
        'r_struct_no_node_score',
        'r_node_corrupt_all_score'
    ]:
        high_scores = [d[score_label] for d in samples]
        avg_gain = np.mean([h - l for h, l in zip(high_scores, low_scores)])
        perc_recover = np.mean([h - l > 0.5 for h, l in zip(high_scores, low_scores)])
        print(f'{score_label}\t{avg_gain:.4f}\t{perc_recover:.4f}')
    print()

Text match: exact (551)
[Example] How many singers do we have?; structed knowledge: | concert_singer | stadium : stadium_id , location , name , capacity , highest , lowest , average | singer : singer_id , name , country , song_name , song_release_year , age , is_male | concert : concert_id , concert_name , theme , stadium_id , year | singer_in_concert : concert_id , singer_id
[Example] select count(*) from *singer*
Score	avg_gain	perc_recover
base_score	0.8927	1.0000
r_text_score	0.4673	0.5245
r_struct_score	0.8768	0.9800
r_node_score	0.7754	0.8621
r_struct_no_node_score	0.3959	0.4410
r_node_corrupt_all_score	0.8129	0.9129

Text match: partial (98)
[Example] Show the name and theme for all concerts and the number of singers in each concert.; structed knowledge: | concert_singer | stadium : stadium_id , location , name , capacity , highest , lowest , average | singer : singer_id , name , country , song_name , song_release_year , age , is_male | concert : concert_id , concert_name , them

In [150]:
ex = samples_by_text_match['no-match'][50]

In [202]:
text_in = ex['text_in']
struct_in = ex['struct_in']

enc_sentence = f"{text_in} The database is as following: {struct_in}"

f"{enc_sentence} => {ex['seq_out']}"

"What are the names of all European countries with at least 3 manufacturers? The database is as following: | car_1 | continents : contid , continent ( europe ) | countries : countryid , countryname , continent | car_makers : id , maker , fullname , country | model_list : modelid , maker , model | car_names : makeid , model , make | cars_data : id , mpg , cylinders , edispl , horsepower , weight , accelerate , year => select t1.countryname from countries as t1 join continents as t2 on t1.continent = t2.contid join car_makers as t3 on t1.countryid = t3.country where t2.continent = 'europe' group by t1.countryname having count(*) >= 3;"

In [203]:
ex['ex_id']

130

### Exp-2.2: dirty text struct restore

#### Load & Plotting results

In [382]:
expect_type = 'column'

res_json_path = f'/home/yshao/Projects/rome/results/exp2.2_dirty_text_struct_restore/exp=2.2_dev_{expect_type}.jsonl'

with open(res_json_path, 'r') as f:
    all_samples = [json.loads(l) for l in f]

good_trace_results = []
for ex in all_samples:
    for r in ex['trace_results']:
        if r['is_good_sample']:
            r['ex_id'] = ex['ex_id']
            good_trace_results.append(r)
    
len(all_samples), len(good_trace_results)

(1034, 1084)

In [383]:
good_trace_results[0].keys()

dict_keys(['enc_sentence', 'seq_out', 'dec_prompt', 'expect', 'expect_type', 'db_id', 'expect_input_ranges', 'expect_table', 'category', 'low_score', 'high_score', 'input_ids', 'input_tokens', 'dec_input_ids', 'dec_input_tokens', 'subject_range', 'subject_range_individual_indices', 'answer', 'window', 'correct_prediction', 'kind', 'sever_kind', 'scores', 'is_good_sample', 'part', 'ex_id'])

In [239]:
# # Generating all plots
# fig_save_dir = f'/home/yshao/Projects/rome/results/figs/exp2.2_dirty_text_struct_restore/exp=2.2_dev_column'

# for i, r in enumerate(tqdm(good_trace_results)):
#     result = dict(r)

#     enc_s, dec_s = result['scores']
    
#     # enc_s = np.array(enc_s)
#     enc_layers_vec = np.array(enc_s[0])
#     assert enc_layers_vec.shape == (mt_uskg.num_enc_layers,)
#     enc_token_range, = result['enc_token_range']
    
#     enc_s = np.zeros((len(result['input_tokens']), mt_uskg.num_enc_layers))
#     enc_s[enc_token_range] = enc_layers_vec
    
#     dec_s = np.zeros((len(result['dec_input_tokens']), mt_uskg.num_dec_layers))
#     result['scores'] = [enc_s, dec_s]

#     ex_id = r['ex_id']
#     ctu.plot_trace_heatmap_t5(result, savepdf=os.path.join(fig_save_dir, f'{i}-ex_id={ex_id}.pdf'))
    

#### Check avg per layer

In [240]:
scores_per_layer = [[] for _ in range(mt_uskg.num_enc_layers)]

for r in good_trace_results:
    enc_s, dec_s = r['scores']
    for l, s in enumerate(enc_s[0]):
        scores_per_layer[l].append(s)

In [241]:
all_avg_by_layer = [np.mean(scores) for scores in scores_per_layer]
all_avg_by_layer

[0.04998245431660366,
 0.05325091241746874,
 0.05578506940298435,
 0.056500998754504546,
 0.05632673384162137,
 0.060122467615442834,
 0.0743413356685932,
 0.07793125936657581,
 0.1187241473144563,
 0.1544875932789873,
 0.18513770163235047,
 0.21917309280363884,
 0.28553838302240797,
 0.3615626793599896,
 0.3617946804873726,
 0.3946494784126291,
 0.47267543055045763,
 0.44948675620885103,
 0.4737027594501244,
 0.5257112130350594,
 0.5352272429749859,
 0.6249380804599364,
 0.670510063510466,
 0.7290238733215495]

In [242]:
print('Layer\tAvg. Score')
for l, avg in enumerate(all_avg_by_layer):
    print(f'{l:<5d}\t{avg:.4f}')

Layer	Avg. Score
0    	0.0500
1    	0.0533
2    	0.0558
3    	0.0565
4    	0.0563
5    	0.0601
6    	0.0743
7    	0.0779
8    	0.1187
9    	0.1545
10   	0.1851
11   	0.2192
12   	0.2855
13   	0.3616
14   	0.3618
15   	0.3946
16   	0.4727
17   	0.4495
18   	0.4737
19   	0.5257
20   	0.5352
21   	0.6249
22   	0.6705
23   	0.7290


In [ ]:
fig, ax = plt.subplots(1, 1)
ax.plot(all_avg_by_layer, label='All')
ax.set_xlabel('Layer to restore')
ax.set_ylabel('Avg. P(correct answer)')
ax.legend()

####  by hardness 

In [244]:
# {h : [[score], ...]}
scores_by_hardness = {hardness : [] for hardness in ['easy', 'medium', 'hard', 'extra']}   

for r in good_trace_results:
    enc_s, dec_s = r['scores']
    scores, = enc_s
#     hardness = spider_id2hardness[r['ex_id']]
    hardness = r['category']['sql_hardness']
    scores_by_hardness[hardness].append(scores)

[(h, len(scores)) for h, scores in scores_by_hardness.items()]

[('easy', 191), ('medium', 516), ('hard', 216), ('extra', 161)]

In [245]:
for h, sample_scores in scores_by_hardness.items():
    print(h)
    scores_by_layer = zip(*sample_scores)
    avg_by_layer = [np.mean(scores) for scores in scores_by_layer]
    print([f'{avg:.2f}' for avg in avg_by_layer])
    print()

easy
['0.05', '0.05', '0.06', '0.06', '0.06', '0.06', '0.08', '0.09', '0.14', '0.20', '0.24', '0.29', '0.35', '0.41', '0.40', '0.43', '0.49', '0.48', '0.50', '0.54', '0.55', '0.65', '0.71', '0.78']

medium
['0.05', '0.05', '0.06', '0.06', '0.06', '0.06', '0.07', '0.07', '0.11', '0.14', '0.17', '0.20', '0.27', '0.35', '0.35', '0.38', '0.45', '0.42', '0.45', '0.50', '0.51', '0.60', '0.65', '0.71']

hard
['0.04', '0.05', '0.05', '0.05', '0.05', '0.05', '0.06', '0.07', '0.11', '0.14', '0.17', '0.21', '0.27', '0.35', '0.37', '0.41', '0.49', '0.47', '0.48', '0.54', '0.54', '0.60', '0.65', '0.71']

extra
['0.06', '0.06', '0.07', '0.07', '0.07', '0.07', '0.09', '0.10', '0.13', '0.16', '0.19', '0.22', '0.28', '0.35', '0.35', '0.38', '0.50', '0.48', '0.52', '0.58', '0.60', '0.70', '0.72', '0.74']



In [ ]:
fig, ax = plt.subplots()
ax.plot(all_avg_by_layer, label='All')
for h, sample_scores in scores_by_hardness.items():
    h_scores_by_layer = zip(*sample_scores)
    h_avg_by_layer = [np.mean(scores) for scores in h_scores_by_layer]
    ax.plot(h_avg_by_layer, label=h)
ax.set_xlabel('Layer to restore')
ax.set_ylabel('Avg. P(correct answer)')
ax.legend()

####  by node role 

In [247]:
# {h : [[score], ...]}
scores_by_node_role = defaultdict(list)

for r in good_trace_results:
    enc_s, dec_s = r['scores']
    scores, = enc_s
#     role = _detect_node_role(' '.join(r['dec_input_tokens']))
    role = r['category']['node_role']
    scores_by_node_role[role].append(scores)

[(r, len(scores)) for r, scores in scores_by_node_role.items()]

[('select', 642),
 ('order by', 73),
 ('where', 267),
 ('group by', 26),
 ('join', 62),
 ('having', 4),
 ('from', 10)]

In [248]:
for r, sample_scores in scores_by_node_role.items():
    if len(sample_scores) < 10:
        continue
    print(r)
    scores_by_layer = zip(*sample_scores)
    avg_by_layer = [np.mean(scores) for scores in scores_by_layer]
    print([f'{avg:.2f}' for avg in avg_by_layer])
    print()

select
['0.03', '0.03', '0.03', '0.03', '0.03', '0.03', '0.04', '0.04', '0.04', '0.06', '0.07', '0.09', '0.13', '0.18', '0.18', '0.20', '0.28', '0.26', '0.29', '0.35', '0.37', '0.54', '0.62', '0.72']

order by
['0.05', '0.05', '0.05', '0.05', '0.05', '0.05', '0.06', '0.06', '0.07', '0.09', '0.13', '0.17', '0.29', '0.50', '0.49', '0.59', '0.71', '0.63', '0.65', '0.74', '0.75', '0.76', '0.75', '0.77']

where
['0.08', '0.09', '0.10', '0.10', '0.10', '0.12', '0.16', '0.17', '0.30', '0.40', '0.47', '0.54', '0.65', '0.75', '0.76', '0.81', '0.89', '0.87', '0.87', '0.91', '0.91', '0.88', '0.88', '0.88']

group by
['0.15', '0.16', '0.18', '0.18', '0.18', '0.18', '0.20', '0.21', '0.30', '0.40', '0.46', '0.50', '0.65', '0.74', '0.71', '0.77', '0.87', '0.81', '0.85', '0.88', '0.86', '0.80', '0.85', '0.88']

join
['0.06', '0.06', '0.06', '0.06', '0.06', '0.06', '0.06', '0.06', '0.06', '0.07', '0.07', '0.09', '0.15', '0.22', '0.23', '0.25', '0.33', '0.30', '0.31', '0.33', '0.30', '0.27', '0.26', '0.

In [ ]:
fig, ax = plt.subplots()
ax.plot(all_avg_by_layer, label='All')
for r, sample_scores in scores_by_node_role.items():
    if len(sample_scores) < 50:
        continue
    r_scores_by_layer = zip(*sample_scores)
    r_avg_by_layer = [np.mean(scores) for scores in r_scores_by_layer]
    ax.plot(r_avg_by_layer, label=r)
ax.set_xlabel('Layer to restore')
ax.set_ylabel('Avg. P(correct answer)')
ax.legend()

#### by text matching

In [250]:
scores_by_text_matching = {k: [] for k in ['exact', 'partial', 'no-match']}

for r in good_trace_results:
#     spider_ex = processed_spider_dev[r['ex_id']]
#     col = r['target_node']
#     tab = r['target_node_table']
#     m = ctu.check_text_match(spider_ex, col, tab)
    m = r['category']['text_match']

    enc_s, dec_s = r['scores']
    scores, = enc_s
    scores_by_text_matching[m].append(scores)

[(m, len(samples)) for m, samples in scores_by_text_matching.items()]

[('exact', 613), ('partial', 170), ('no-match', 301)]

In [251]:
for r, sample_scores in scores_by_text_matching.items():
    if len(sample_scores) < 10:
        continue
    print(r)
    scores_by_layer = zip(*sample_scores)
    avg_by_layer = [np.mean(scores) for scores in scores_by_layer]
    print([f'{avg:.2f}' for avg in avg_by_layer])
    print()

exact
['0.05', '0.06', '0.06', '0.06', '0.06', '0.06', '0.08', '0.08', '0.13', '0.17', '0.20', '0.24', '0.29', '0.35', '0.35', '0.38', '0.46', '0.43', '0.45', '0.51', '0.53', '0.63', '0.68', '0.75']

partial
['0.05', '0.05', '0.05', '0.05', '0.05', '0.05', '0.05', '0.06', '0.07', '0.09', '0.12', '0.14', '0.22', '0.32', '0.32', '0.36', '0.44', '0.44', '0.47', '0.52', '0.52', '0.62', '0.67', '0.73']

no-match
['0.05', '0.05', '0.05', '0.05', '0.05', '0.06', '0.08', '0.08', '0.13', '0.15', '0.19', '0.23', '0.31', '0.41', '0.41', '0.44', '0.52', '0.50', '0.52', '0.57', '0.56', '0.61', '0.65', '0.70']



In [ ]:
fig, ax = plt.subplots()
ax.plot(all_avg_by_layer, label='All')
for m, sample_scores in scores_by_text_matching.items():
    if len(sample_scores) < 10:
        continue
    r_scores_by_layer = zip(*sample_scores)
    r_avg_by_layer = [np.mean(scores) for scores in r_scores_by_layer]
    ax.plot(r_avg_by_layer, label=m)
ax.set_xlabel('Layer to restore')
ax.set_ylabel('Avg. P(correct answer)')
ax.legend()

### Exp-3.1: dirty struct context restore

#### Load & Check results

In [65]:
res_path = '/home/yshao/Projects/rome/results/exp3.1_dirty_struct_context_restore/exp=3.1_dev_table-tmp.jsonl'

with open(res_path, 'r') as f:
    all_samples = [json.loads(l) for l in f]
len(all_samples)

10

In [66]:
all_samples[0]

{'ex_id': 0,
 'trace_results': [{'enc_sentence': 'How many singers do we have?; structed knowledge: | concert_singer | stadium : stadium_id , location , name , capacity , highest , lowest , average | singer : singer_id , name , country , song_name , song_release_year , age , is_male | concert : concert_id , concert_name , theme , stadium_id , year | singer_in_concert : concert_id , singer_id',
   'seq_out': 'select count(*) from singer',
   'dec_prompt': 'select count(*) from',
   'expect': 'singer',
   'expect_type': 'table',
   'db_id': 'concert_singer',
   'expect_input_ranges': [[47, 48]],
   'expect_table': 'singer',
   'answer': 'singer',
   'base_score': 0.9999998807907104,
   'answers_t': [7634],
   'correct_prediction': True,
   'category': {'sql_hardness': 'easy',
    'node_role': 'from',
    'text_match': 'exact'},
   'self_ranges': [[46, 50]],
   'struct_context_ranges': [[15, 46], [50, 125]],
   'trace_scores': {'single_layer_corrupt': {},
    'low_layers_restore': {},
   

In [68]:
total_samples = 0
n_good_samples = 0
n_too_hard = 0      # wrong answer 
n_too_easy = 0      # base - low < 0.5

In [69]:
good_samples = []
bad_samples = []

for i, ex in enumerate(all_samples):
    for d in ex['trace_results']:
        total_samples += 1
        if d['is_good_sample']:
            n_good_samples += 1
            d['ex_id'] = i
            good_samples.append(d)
        elif not d['correct_prediction']:
            n_too_hard += 1
            bad_samples.append(d)
        else:
            assert d['base_score'] - d['low_score'] < 0.5
            n_too_easy += 1
            bad_samples.append(d)
            
total_samples, (n_good_samples, len(good_samples)), n_too_hard, n_too_easy, len(bad_samples)

(17, (0, 0), 0, 17, 17)

In [158]:
for d in good_samples:
    print(d['enc_sentence'])
    print(d['dec_prompt'], '-->', d['expect'])
    print(json.dumps(d['trace_scores'], indent=1))

In [ ]:
for d in bad_samples:
    print(d['enc_sentence'])
    print(d['dec_prompt'], '-->', d['expect'])
    print()

### Exp-3.2: dirty struct context compare

#### Load & Check results

In [221]:
res_path = '/home/yshao/Projects/rome/results/exp3.2_dirty_struct_context_compare/exp=3.2_dev_table.jsonl'

with open(res_path, 'r') as f:
    all_samples = [json.loads(l) for l in f]
len(all_samples)

1034

In [222]:
all_samples[0]

{'ex_id': 0,
 'trace_results': [{'enc_sentence': 'How many singers do we have?; structed knowledge: | concert_singer | stadium : stadium_id , location , name , capacity , highest , lowest , average | singer : singer_id , name , country , song_name , song_release_year , age , is_male | concert : concert_id , concert_name , theme , stadium_id , year | singer_in_concert : concert_id , singer_id',
   'seq_out': 'select count(*) from singer',
   'dec_prompt': 'select count(*) from',
   'expect': 'singer',
   'expect_type': 'table',
   'db_id': 'concert_singer',
   'expect_input_ranges': [[47, 48]],
   'expect_table': 'singer',
   'answer': 'singer',
   'base_score': 0.9999998807907104,
   'answers_t': [7634],
   'correct_prediction': True,
   'category': {'sql_hardness': 'easy',
    'node_role': 'from',
    'text_match': 'exact'},
   'self_ranges': [[46, 50]],
   'struct_context_ranges': [[15, 46], [50, 125]],
   'scc_score': 1.0,
   'scc_answers_t': [7634],
   'scc_answer': 'singer',
   's

In [223]:
correctness_compare_counter = Counter([d['compare']['correctness_compare'] for ex in all_samples for d in ex['trace_results']])
correctness_compare_counter

Counter({'both': 1424, 'clean+': 123, 'scc+': 105, 'none': 31})

In [ ]:
for i, ex in enumerate(all_samples):
    for d in ex['trace_results']:
        _comp = d['compare']['correctness_compare']
#         if _comp in {'scc+', 'clean+'}:
        if (_comp == 'scc+') and (d['category']['node_role'] != 'join'):
            print(f'#{i}:', d['enc_sentence'])
            print('SQL:', d['dec_prompt'], '-->', d['expect'])
            print('Clean:', d['answer'])
            print('SCC:', d['scc_answer'])
            print('COMP:', d['compare']['correctness_compare'])
            print()

In [226]:
_clean_node_role_cnt = Counter()
# TODO: make a larger counter: {comp -> category_k -> category_v -> count}

for i, ex in enumerate(all_samples):
    for d in ex['trace_results']:
        if d['compare']['correctness_compare'] == 'clean+':
            _role = d['category']['node_role']
            _clean_node_role_cnt[_role] += 1

_clean_node_role_cnt

Counter({'join': 83, 'from': 40})

In [227]:
_scc_node_role_cnt = Counter()

for i, ex in enumerate(all_samples):
    for d in ex['trace_results']:
        if d['compare']['correctness_compare'] == 'scc+':
            _role = d['category']['node_role']
            _scc_node_role_cnt[_role] += 1

_scc_node_role_cnt

Counter({'from': 73, 'join': 32})

### Exp-4: inspect attention

In [131]:
exp_type = 'column'

res_path = f'/home/yshao/Projects/rome/results/exp4_inspect_attention/exp=4_dev_{exp_type}_both.jsonl'

with open(res_path, 'r') as f:
    all_samples = [json.loads(l) for l in f]
len(all_samples)

104

In [132]:
all_samples[1]['trace_results'][0].keys()

dict_keys(['enc_sentence', 'seq_out', 'dec_prompt', 'expect', 'expect_type', 'db_id', 'expect_input_ranges', 'expect_table', 'category', 'answer', 'probs', 'base_score', 'answers_t', 'correct_prediction', 'attentions'])

In [133]:
d = all_samples[6]['trace_results'][0]
d['enc_sentence'], d['dec_prompt'], d['expect']

("What are the students' first names who have both cats and dogs as pets?; structed knowledge: | pets_1 | student : stuid , lname , fname , age , sex , major , advisor , city_code | has_pet : stuid , petid | pets : petid , pettype ( dog , cat ) , pet_age , weight",
 'select t1.',
 'fname')

In [134]:
d.keys(), d['attentions'].keys(), d['attentions']['enc'].keys()

(dict_keys(['enc_sentence', 'seq_out', 'dec_prompt', 'expect', 'expect_type', 'db_id', 'expect_input_ranges', 'expect_table', 'category', 'answer', 'probs', 'base_score', 'answers_t', 'correct_prediction', 'attentions']),
 dict_keys(['enc', 'cross', 'dec']),
 dict_keys(['attn', 'head_tokens', 'cand_tokens']))

#### Plotting

In [124]:
def _draw_single_plot_2(ax, val_mat, x_labels=None, y_labels=None, title=None):
    """
    X: # heads
    Y: cand tokens
    Title: head token (together with full expect)
    """
    if isinstance(val_mat, list):
        val_mat = numpy.array(val_mat)
    h = ax.pcolormesh(
        val_mat,
        cmap="Reds",
        vmax=1.0,
        vmin=0.0,
    )
    ax.invert_yaxis()
    ax.set_yticks([0.5 + i for i in range(val_mat.shape[0])])
    ax.set_xticks([0.5 + i for i in range(val_mat.shape[1])])
    if x_labels is not None:
        ax.set_xticklabels(x_labels, fontsize=8)
    if y_labels is not None:
        ax.set_yticklabels(y_labels, fontsize=8)

    if title is not None:
        ax.set_title(title)
    ax.set_xlabel(f"# Head")
    ax.set_ylabel(f"Attention candidate tokens")
    
    # cb = plt.colorbar(h)
    # divider = make_axes_locatable(ax)
    # cax = divider.append_axes('right', size='5%', pad=0.05)
    # cb = fig.colorbar(h, cax=cax)
    cb = plt.colorbar(h, ax=ax)

#     if xlabel is not None:
#         ax.set_xlabel(xlabel)
#     elif answer is not None:
#         # The following should be cb.ax.set_xlabel, but this is broken in matplotlib 3.5.1.
#         cb.ax.set_title(f"p({str(answer).strip()})", y=-0.16, fontsize=10)

In [127]:
def plot_uskg_attention(d, att_part, inspect_layers=None, savepdf=None):
    """
    Assume 16 heads, 24 layers (T5 large config)
    
    att_part: enc, cross, dec 
    """
    
    ## encoder self attention 
    if inspect_layers is None:
        inspect_layers = [0, 6, 12, 18, 23]
    elif inspect_layers == 'all':
        inspect_layers = [i for i in range(24)]
    att_dict = d['attentions'][att_part]
    
    cand_len = len(att_dict['cand_tokens'])
    head_len = len(att_dict['head_tokens'])
    prompt_tokens = d['attentions']['dec']['cand_tokens']
    prompt_len = len(prompt_tokens)

    fig_w = len(inspect_layers) * 4 + 2
    fig_h = (0.11*cand_len + 1) * head_len + 2
    fig, ax_list = plt.subplots(
        nrows=head_len,
        ncols=len(inspect_layers),
        squeeze=False,
        figsize=(fig_w, fig_h))

    att_mat = ctu.nested_list_processing(att_dict['attn'], func=float)
    att_mat = np.array(att_mat)
    
    for expect_i in range(len(att_dict['head_tokens'])):
        for l_id, layer in enumerate(inspect_layers):
            val_mat = att_mat[layer, :, expect_i, :]  # layer, all heads, expect tok i -> all toks 
            val_mat = val_mat.transpose()    # (cand_toks, n_heads)
            x_labels = range(val_mat.shape[1])
            y_labels = att_dict['cand_tokens']
            if att_part == 'enc':
                # enc: correct tokens
                title_toks = att_dict['head_tokens'][:expect_i] + [f"*{att_dict['head_tokens'][expect_i]}*"]
            else:
                # cross / dec: use gold tokens from dec_prompt for previous steps and predicted token at this step
                # (dec_prompt ends with the first (head_len-1) tokens of the target node)
                title_toks = prompt_tokens[prompt_len - (head_len-1) : prompt_len - (head_len-1) + expect_i] + [f"*{att_dict['head_tokens'][expect_i]}*"]
            
            title = f"L{layer}  Head token: {' '.join(title_toks)}\n"
            
            ax = ax_list[expect_i, l_id]
            _draw_single_plot_2(ax,
                                val_mat=val_mat, 
                                x_labels=x_labels, 
                                y_labels=y_labels,
                                title=title)
            
    fig.tight_layout()
    if savepdf:
        plt.savefig(savepdf, bbox_inches="tight")
        plt.close()
    else:
        plt.show()

In [118]:
# savepdf_tmpl = '/home/yshao/Projects/rome/results/figs/exp4_inspect_attention/tmp-6-{}.pdf'
# for att_part in ['enc', 'cross', 'dec']:
#     plot_uskg_attention(d, att_part, savepdf=savepdf_tmpl.format(att_part))

In [119]:
## Run for all! 

fig_dir  = f'/home/yshao/Projects/rome/results/figs/exp4_inspect_attention/dev_{exp_type}'
os.makedirs(fig_dir, exist_ok=True)

global_ex_id = 0
for ex_id in tqdm(range(len(all_samples))):
    for a_ex_id in range(len(all_samples[ex_id]['trace_results'])):
        d = all_samples[ex_id]['trace_results'][a_ex_id]
        for att_part in ['enc', 'cross', 'dec']:
            savepdf_path = os.path.join(fig_dir, f'{global_ex_id}-ex={ex_id}.{a_ex_id}-{att_part}.pdf')
            plot_uskg_attention(d, att_part, savepdf=savepdf_path)
        # print(f'{global_ex_id}-ex={ex_id}.{a_ex_id}')
        global_ex_id += 1

  0%|          | 0/104 [00:00<?, ?it/s]

In [135]:
_sel_samples = []

for ex_id in (range(len(all_samples))):
    for a_ex_id in range(len(all_samples[ex_id]['trace_results'])):
        d = all_samples[ex_id]['trace_results'][a_ex_id]
        if d['category']['text_match'] == 'no-match':
            _sel_samples.append((ex_id, a_ex_id))

print(_sel_samples)

[(5, 1), (5, 2), (6, 0), (6, 1), (6, 2), (6, 3), (9, 0), (9, 1), (9, 2), (10, 0), (10, 1), (11, 0), (12, 2), (13, 1), (13, 2), (13, 3), (13, 4), (15, 0), (17, 1), (20, 0), (20, 1), (21, 0), (21, 1), (21, 2), (22, 0), (22, 1), (22, 2), (22, 3), (23, 0), (24, 0), (24, 1), (24, 2), (24, 3), (25, 0), (25, 1), (39, 0), (39, 1), (45, 0), (45, 1), (47, 2), (55, 2), (55, 3), (55, 4), (58, 0), (58, 1), (61, 0), (63, 0), (63, 1), (64, 2), (67, 1), (71, 1), (72, 0), (72, 1), (73, 1), (74, 0), (75, 0), (75, 2), (76, 0), (77, 0), (77, 1), (77, 2), (79, 0), (79, 1), (80, 0), (80, 1), (81, 1), (81, 2), (82, 1), (85, 0), (85, 1), (89, 0), (89, 1), (90, 0), (90, 1), (91, 0), (94, 0), (96, 0), (96, 1)]


In [137]:
## Run for separate samples 

fig_dir  = f'/home/yshao/Projects/rome/results/figs/exp4_inspect_attention/detail_subset'
os.makedirs(fig_dir, exist_ok=True)

ex_id = 17
a_ex_id = 1
d = all_samples[ex_id]['trace_results'][a_ex_id]
for att_part in ['enc']:
    savepdf_path = os.path.join(fig_dir, f'{exp_type}-ex={ex_id}.{a_ex_id}-{att_part}.pdf')
    plot_uskg_attention(d, att_part, inspect_layers='all', savepdf=savepdf_path)


#### Exp-4.0.1: Prefix attention samples
- Check the samples with high attention on a given layer & head & prefix token
- Haven't found any insights...

In [63]:
total_a_ex = 0
total_tok_sample = 0

for ex_id in (range(len(all_samples))):
    for a_ex_id in range(len(all_samples[ex_id]['trace_results'])):
        d = all_samples[ex_id]['trace_results'][a_ex_id]
        _expect_len = len([i for s, e in d['expect_input_ranges'] for i in range(s, e)])
        
        total_a_ex += 1
        total_tok_sample += _expect_len

total_a_ex, total_tok_sample

(198, 680)

In [57]:
d = all_samples[6]['trace_results'][0]
d['enc_sentence'], d['dec_prompt'], d['expect']

("What are the students' first names who have both cats and dogs as pets?; structed knowledge: | pets_1 | student : stuid , lname , fname , age , sex , major , advisor , city_code | has_pet : stuid , petid | pets : petid , pettype ( dog , cat ) , pet_age , weight",
 'select t1.',
 'fname')

In [61]:
_expect_len = len([i for s, e in d['expect_input_ranges'] for i in range(s, e)])
d['expect_input_ranges'], _expect_len

([[41, 44]], 3)

In [52]:
enc_prefix_att_samples = defaultdict(list)  # [(l_id, head_id, prefix_id) -> [dict(ex_id, a_ex_id, tok_id, ...)]]

for ex_id in tqdm(range(len(all_samples))):
    for a_ex_id in range(len(all_samples[ex_id]['trace_results'])):
# for ex_id in [6]:
#     for a_ex_id in [0]:
        d = all_samples[ex_id]['trace_results'][a_ex_id]
        
        att_mat = ctu.nested_list_processing(d['attentions']['enc']['attn'], func=float)
        # att_mat: (n_layers, n_heads, n_expect_toks, n_input_toks)
        att_mat = np.array(att_mat)
        n_layers, n_heads, n_expect_toks, n_input_toks = att_mat.shape
        
        # (n_layers, n_heads, expect_toks)
#         input_att_positions = np.argmax(att_mat, axis=-1)
#         for l_id in range(n_layers):
#             for h_id in range(n_heads):
#                 for tok_id in range(n_expect_toks):
#                     _max_att_p = input_att_positions[l_id, h_id, tok_id]

        input_att_positions = np.where(att_mat > 0.5)
        for l_id, h_id, tok_id, _max_att_p in zip(*input_att_positions):
            if _max_att_p < 10:
                # is prefix 
                enc_prefix_att_samples[(l_id, h_id, _max_att_p)].append({
                    'ex_id': ex_id,
                    'a_ex_id': a_ex_id,
                    'expect_tok_id': tok_id,
                    'enc_sentence': d['enc_sentence'],
                    'dec_prompt': d['dec_prompt'],
                    'expect': d['expect'],
                })

  0%|          | 0/104 [00:00<?, ?it/s]

In [53]:
len(enc_prefix_att_samples)

1873

In [ ]:
len_profile_counter = Counter([len(v) for k, v in enc_prefix_att_samples.items()])
sorted(len_profile_counter.items())

In [55]:
for k, v in enc_prefix_att_samples.items():
    if len(v) > 120:
        print(k, len(v))

(18, 5, 4) 192
(18, 6, 2) 145
(20, 0, 7) 136
(20, 4, 8) 198
(23, 7, 0) 125
(8, 5, 0) 140
(15, 5, 4) 235
(17, 1, 9) 220
(19, 10, 1) 177
(21, 12, 5) 131
(3, 4, 8) 126
(11, 8, 4) 124
(21, 0, 1) 129


In [ ]:
enc_prefix_att_samples[(23, 7, 0)]

### Exp-5.0: dirty attention vector effect 

#### Load & Check results

In [255]:
expect_type = 'table_alias'
res_path = f'/home/yshao/Projects/rome/results/exp5_0_dirty_attention_vector_effect/exp=5_dev_{expect_type}_encoder-attn=self_attn-corrupt=zero.jsonl'

with open(res_path, 'r') as f:
    all_samples = [json.loads(l) for l in f]
len(all_samples)

1034

In [256]:
total_samples = 0
n_good_samples = 0
n_too_hard = 0      # wrong answer 
n_too_easy = 0      # base - low < 0.5

In [257]:
good_samples = []
bad_samples = []

for i, ex in enumerate(all_samples):
    for d in ex['trace_results']:
        total_samples += 1
#         # TEMP adjustment for column results 
#         d['low_score'] = d['trace_scores']['high_layers_corrupt'].get("0", 0.0)  # "0" is key (for layer 0), 0.0 is default 
#         if d['base_score'] - d['low_score'] < 0.5:
#             d['is_good_sample'] = False
#         # END_TEMP
        if d['is_good_sample']:
            n_good_samples += 1
            d['ex_id'] = i
            good_samples.append(d)
        elif not d['correct_prediction']:
            n_too_hard += 1
            bad_samples.append(d)
        else:
            assert d['base_score'] - d['low_score'] < 0.5, (i, d)
            n_too_easy += 1
            bad_samples.append(d)
            
total_samples, (n_good_samples, len(good_samples)), n_too_hard, n_too_easy, len(bad_samples), \
n_good_samples + n_too_easy

(2039, (164, 164), 339, 1536, 1875, 1700)

In [ ]:
[s for s in bad_samples if s['correct_prediction']][0]

In [ ]:
good_samples[0]

#### Overall avg

In [193]:
trace_scores_avg = {k: {str(l): 0 for l in range(24)} for k in good_samples[0]['trace_scores'].keys()}

In [194]:
for d in good_samples:
    for k, layer_d in d['trace_scores'].items():
        for l, s in layer_d.items():
            trace_scores_avg[k][l] += s

for k, layer_d in trace_scores_avg.items():
    for l, s in layer_d.items():
        layer_d[l] = s / len(good_samples)

In [ ]:
trace_scores_avg

#### Avg by aspects (category)
- Still kind of linear, as in exp-2.2

In [243]:
d['category']

{'sql_hardness': 'hard', 'node_role': 'where', 'text_match': 'partial'}

In [265]:
# Key: (trace_key, aspect, asp_val, layer) -> [scores]
trace_scores_by_aspect = defaultdict(lambda: defaultdict(lambda: defaultdict(lambda: defaultdict(list))))
trace_scores_avg_by_aspect = defaultdict(lambda: defaultdict(lambda: defaultdict(lambda: defaultdict(float))))
trace_scores_cnt_by_aspect = defaultdict(lambda: defaultdict(int))  # no trace key & layer key 

In [266]:
for d in good_samples:
    for trace_k, trace_layer_d in d['trace_scores'].items():
        for aspect, asp_val in d['category'].items():
            for l, s in trace_layer_d.items():
                trace_scores_by_aspect[trace_k][aspect][asp_val][l].append(s)

for trace_k, d1 in trace_scores_by_aspect.items():
    for asp_k, d2 in d1.items():
        for asp_v, d3 in d2.items():
            for l, s in d3.items():
                trace_scores_avg_by_aspect[trace_k][asp_k][asp_v][l] = np.mean(s)
                trace_scores_cnt_by_aspect[asp_k][asp_v] = len(s)

In [267]:
trace_scores_cnt_by_aspect

defaultdict(<function __main__.<lambda>()>,
            {'sql_hardness': defaultdict(int,
                         {'medium': 400,
                          'hard': 155,
                          'easy': 142,
                          'extra': 170}),
             'node_role': defaultdict(int,
                         {'where': 268,
                          'select': 412,
                          'order by': 66,
                          'join': 92,
                          'group by': 25,
                          'having': 4}),
             'text_match': defaultdict(int,
                         {'no-match': 360, 'partial': 148, 'exact': 359})})

In [ ]:
trace_scores_avg_by_aspect['high_layers_corrupt']

### Exp-5.2: attention section removal effect

#### Load & Check

In [679]:
expect_type = 'table'

res_path = f'/home/yshao/Projects/rome/results/exp5_2_attention_section_removal_effect/exp=5.2.1_dev_{expect_type}_encoder-attn=self_attn-corrupt=zero.jsonl'

with open(res_path, 'r') as f:
    all_samples = [json.loads(l) for l in f]
len(all_samples)

1034

In [680]:
total_samples = 0
n_good_samples = 0
n_too_hard = 0      # wrong answer 
n_too_easy = 0      # base - low < 0.5

In [681]:
good_samples = []
bad_samples = []

for i, ex in enumerate(all_samples):
    for d in ex['trace_results']:
        total_samples += 1

        if d['is_good_sample']:
            n_good_samples += 1
            d['ex_id'] = i
            good_samples.append(d)
        elif not d['correct_prediction']:
            n_too_hard += 1
            bad_samples.append(d)
        else:
            assert d['base_score'] - d['low_score'] < 0.5
            n_too_easy += 1
            bad_samples.append(d)
            
total_samples, (n_good_samples, len(good_samples)), n_too_hard, n_too_easy, len(bad_samples),\
f'good / correct = {n_good_samples} / {n_good_samples + n_too_easy}'

(1683, (1207, 1207), 136, 340, 476, 'good / correct = 1207 / 1547')

In [682]:
[s for s in bad_samples if not s['correct_prediction']][0]

{'enc_sentence': 'Find the number of concerts happened in the stadium with the highest capacity .; structed knowledge: | concert_singer | stadium : stadium_id , location , name , capacity , highest , lowest , average | singer : singer_id , name , country , song_name , song_release_year , age , is_male | concert : concert_id , concert_name , theme , stadium_id , year | singer_in_concert : concert_id , singer_id',
 'seq_out': 'select count(*) from concert where stadium_id = (select stadium_id from stadium order by capacity desc limit 1)',
 'dec_prompt': 'select count(*) from',
 'expect': 'concert',
 'expect_type': 'table',
 'db_id': 'concert_singer',
 'expect_input_ranges': [[88, 89]],
 'expect_table': 'concert',
 'answer': 'stadium',
 'base_score': 0.9941871166229248,
 'answers_t': [14939],
 'correct_prediction': False,
 'category': {'sql_hardness': 'hard',
  'node_role': 'from',
  'text_match': 'exact'},
 'self_ranges': [[87, 91]],
 'struct_context_ranges': [[22, 87], [91, 132]],
 'is_

In [ ]:
good_samples[0]

#### Overall avg

In [647]:
trace_scores_avg = {sect_k : defaultdict(int) for sect_k in good_samples[0]['trace_scores'].keys()}

for d in good_samples:
    for sect_k, sect_d in d['trace_scores'].items():
        for k, v in sect_d.items():
            if k == 'window':
                for l, s in v.items():
                    trace_scores_avg[sect_k][f'{k}-{l}'] += s
            else:
                s = v
                trace_scores_avg[sect_k][k] += s

for sect_k, sect_d in trace_scores_avg.items():
    for k, s in sect_d.items():
        sect_d[k] = s / len(good_samples)

In [ ]:
trace_scores_avg

#### Avg by aspects (category)

In [696]:
# TEMP patch for node_len category 
for d in good_samples + bad_samples:
    node_len = len(d['answers_t'])
    assert len(mt_uskg.tokenizer.tokenize(d['expect'])) == node_len, (d['expect'], node_len)
    d['category']['node_len'] = str(node_len) if node_len <= 3 else '4+'

In [701]:
d['category']

{'sql_hardness': 'medium',
 'node_role': 'from',
 'text_match': 'exact',
 'node_len': '1'}

In [651]:
# Key: (sect_k, aspect, asp_val, layer) -> [scores]
trace_scores_by_aspect = defaultdict(lambda: defaultdict(lambda: defaultdict(lambda: defaultdict(list))))
trace_scores_avg_by_aspect = defaultdict(lambda: defaultdict(lambda: defaultdict(lambda: defaultdict(float))))
trace_scores_cnt_by_aspect = defaultdict(lambda: defaultdict(int))  # no sect key & layer key 

In [652]:
for d in good_samples:
    for sect_k, sect_d in d['trace_scores'].items():
        for aspect, asp_val in d['category'].items():
            for k, v in sect_d.items():
                if k == 'window':
                    for l, s in v.items():
                        if not (int(l) % 4 == 3): continue
                        layer_k = f'{k}-{l}'
                        trace_scores_by_aspect[sect_k][aspect][asp_val][layer_k].append(s)
                else:
                    layer_k = k
                    s = v
                    trace_scores_by_aspect[sect_k][aspect][asp_val][layer_k].append(s)
                    
for sect_k, d1 in trace_scores_by_aspect.items():
    for asp_k, d2 in d1.items():
        for asp_v, d3 in d2.items():
            for layer_k, s in d3.items():
                trace_scores_avg_by_aspect[sect_k][asp_k][asp_v][layer_k] = np.mean(s)
                trace_scores_cnt_by_aspect[asp_k][asp_v] = len(s)

In [653]:
for sect_k, sect_d in trace_scores_avg_by_aspect.items():
    sect_d['overall'] = dict()
    for layer_k, s in trace_scores_avg[sect_k].items():
        if layer_k.startswith('window'):
            # only keep a subset of layers 
            _, l = layer_k.split('-')
            if not (int(l) % 4 == 3): continue
        sect_d['overall'][layer_k] = s

In [654]:
trace_scores_cnt_by_aspect

defaultdict(<function __main__.<lambda>()>,
            {'sql_hardness': defaultdict(int,
                         {'medium': 378,
                          'hard': 148,
                          'easy': 134,
                          'extra': 160}),
             'node_role': defaultdict(int,
                         {'where': 248,
                          'select': 393,
                          'order by': 63,
                          'join': 91,
                          'group by': 21,
                          'having': 4}),
             'text_match': defaultdict(int,
                         {'no-match': 345, 'partial': 142, 'exact': 333}),
             'node_len': defaultdict(int,
                         {'1': 329, '3': 238, '4+': 160, '2': 93})})

In [ ]:
trace_scores_avg_by_aspect['self']

In [641]:
dump_d = ctu.nested_json_processing(trace_scores_avg_by_aspect, func=lambda x: np.format_float_positional(x, precision=4, min_digits=4))
# dump_d

In [642]:
dump_path = f'/home/yshao/Projects/rome/results/exp5_2_attention_section_removal_effect/summ-exp=5.2.1_dev_{expect_type}_encoder-attn=self_attn-corrupt=zero.jsonl'

with open(dump_path, 'w') as f:
    json.dump(dump_d, f, indent=1)

#### (one-time temp patch)

In [531]:
# expect_type = 'table_alias'
# orig_res_path = f'/home/yshao/Projects/rome/results/exp5_2_attention_section_removal_effect/no_structcontext-exp=5.2.1_dev_{expect_type}_encoder-attn=self_attn-corrupt=zero.jsonl'
# add_res_path = f'/home/yshao/Projects/rome/results/exp5_2_attention_section_removal_effect/exp=5.2.1+structcontext_dev_{expect_type}_encoder-attn=self_attn-corrupt=zero.jsonl'

# merge_res_path = f'/home/yshao/Projects/rome/results/exp5_2_attention_section_removal_effect/exp=5.2.1_dev_{expect_type}_encoder-attn=self_attn-corrupt=zero.jsonl'

In [532]:
# with open(orig_res_path, 'r') as f:
#     orig_all_samples = [json.loads(l) for l in f]
# with open(add_res_path, 'r') as f:
#     add_all_samples = [json.loads(l) for l in f]

# f = open(merge_res_path, 'w')
    
# for i, (orig_ex, add_ex) in enumerate(zip(orig_all_samples, add_all_samples)):
#     assert len(orig_ex['trace_results']) == len(add_ex['trace_results']), i
#     # There is randomness in the order of expected node (from set()), thus sorting here 
#     orig_ex['trace_results'].sort(key=lambda d: len(d['dec_prompt']))
#     add_ex['trace_results'].sort(key=lambda d: len(d['dec_prompt']))
#     for j, (orig_d, add_d) in enumerate(zip(orig_ex['trace_results'], add_ex['trace_results'])):
#         assert orig_d['is_good_sample'] == add_d['is_good_sample'], (i, j)
#         if not orig_d['is_good_sample']:
#             continue
            
#         # is good sample: add the new sections 
#         orig_d['trace_scores']['struct_context'] = add_d['trace_scores']['struct_context']
#         orig_d['trace_scores']['text+struct_context'] = add_d['trace_scores']['text+struct_context']
        
#     f.write(json.dumps(orig_ex, indent=None) + '\n')
    
# f.close()

#### Single samples observations

In [702]:
good_samples[0]['trace_scores'].keys()

dict_keys(['prefix', 'text', 'struct', 'text+struct', 'all', 'self', 'struct_context', 'text+struct_context'])

In [703]:
_id = 0

d = good_samples[_id]

check_info_d = defaultdict(dict)

for sect_k, sect_d in d['trace_scores'].items():
    for layer_k, s in sect_d.items():
        if layer_k == 'window':
            layer_k = 'window-19'
            s = s['19']
        if s < 0.5:
            check_info_d[sect_k][layer_k] = s

In [704]:
print(json.dumps(check_info_d, indent=2))

{
  "text+struct": {
    "all_layers": 0.3990614414215088
  },
  "all": {
    "all_layers": 0.4772564172744751
  }
}


##### Layer

In [705]:
### Check "breaking" window layer, i.e. those with sudden changes 
### For now: single layer drop > _th

_th = 0.4
check_info_l = []
for i, d in enumerate(good_samples):
#     for sect_k, sect_d in d['trace_scores'].items():
    sect_k = 'all'
    sect_d = d['trace_scores'][sect_k]
    window_d = sect_d['window']
    for l in range(1, 24):
        if window_d[str(l-1)] - window_d[str(l)] > _th:
            _info_d = {
                'id': i,
                'sect_k': sect_k,
                'layer': l,
                'last_layer_score': window_d[str(l-1)],
                'this_layer_score': window_d[str(l)],
            }
            check_info_l.append(_info_d)
            break
len(check_info_l)

844

In [706]:
len(check_info_l), len(good_samples)

(844, 1207)

In [707]:
break_layer_counter = Counter([_d['layer'] for _d in check_info_l])
sorted(break_layer_counter.items())

[(1, 10),
 (2, 25),
 (3, 31),
 (4, 81),
 (5, 24),
 (6, 3),
 (7, 15),
 (8, 87),
 (9, 69),
 (10, 19),
 (11, 38),
 (12, 36),
 (13, 70),
 (14, 101),
 (15, 27),
 (16, 31),
 (17, 31),
 (18, 79),
 (19, 67)]

In [ ]:
for info_d in check_info_l:
    if info_d['layer'] < 6:
        print(info_d)
        sample_id = info_d['id']
        d = good_samples[sample_id]
        print(d['enc_sentence'])
        print(d['dec_prompt'], '---->', d['expect'])
        print('Categories:', d['category'])
        print('--' * 20)

In [713]:
sample_counter_by_aspect = defaultdict(Counter)  # [asp_k, asp_v] -> count 
sample_counter = Counter()

for info_d in check_info_l:
    if info_d['layer'] < 6:
        sample_id = info_d['id']
        d = good_samples[sample_id]
        text_match = d['category']['text_match']
        node_len = d['category']['node_len']
        sample_counter[(text_match, node_len)] += 1
        
        for asp_k, asp_v in d['category'].items():
            sample_counter_by_aspect[asp_k][asp_v] += 1

In [715]:
sample_counter_by_aspect

defaultdict(collections.Counter,
            {'sql_hardness': Counter({'medium': 58,
                      'hard': 42,
                      'extra': 51,
                      'easy': 20}),
             'node_role': Counter({'from': 105, 'join': 66}),
             'text_match': Counter({'partial': 59,
                      'no-match': 74,
                      'exact': 38}),
             'node_len': Counter({'4+': 60, '3': 51, '2': 24, '1': 36})})

In [716]:
sample_counter.most_common()

[(('partial', '3'), 30),
 (('partial', '4+'), 29),
 (('exact', '1'), 22),
 (('no-match', '3'), 21),
 (('no-match', '4+'), 21),
 (('no-match', '2'), 18),
 (('no-match', '1'), 14),
 (('exact', '4+'), 10),
 (('exact', '2'), 6)]

In [717]:
sample_counter_by_aspect = defaultdict(Counter)  # [asp_k, asp_v] -> count 
sample_counter = Counter()

for info_d in check_info_l:
    if info_d['layer'] > 18:
        sample_id = info_d['id']
        d = good_samples[sample_id]
        text_match = d['category']['text_match']
        node_len = d['category']['node_len']
        sample_counter[(text_match, node_len)] += 1
        
        for asp_k, asp_v in d['category'].items():
            sample_counter_by_aspect[asp_k][asp_v] += 1

In [718]:
sample_counter_by_aspect

defaultdict(collections.Counter,
            {'sql_hardness': Counter({'hard': 9,
                      'medium': 27,
                      'easy': 11,
                      'extra': 20}),
             'node_role': Counter({'join': 21, 'from': 46}),
             'text_match': Counter({'exact': 46,
                      'no-match': 20,
                      'partial': 1}),
             'node_len': Counter({'3': 12, '1': 46, '2': 6, '4+': 3})})

In [719]:
sample_counter.most_common()

[(('exact', '1'), 38),
 (('no-match', '3'), 9),
 (('no-match', '1'), 8),
 (('exact', '3'), 3),
 (('no-match', '2'), 3),
 (('exact', '2'), 3),
 (('exact', '4+'), 2),
 (('partial', '4+'), 1)]

In [743]:
# _min_p = 1.0

# for i, d in enumerate(good_samples):
#     sect_k = 'text'
#     sect_d = d['trace_scores'][sect_k]
#     _min_p = min(_min_p, sect_d['all_layers'])

# _min_p

7.385318918917694e-12

In [751]:
### Systematic 

ob_sect_k = 'all'

_th = 0.4

check_info_l = []
all_layers_eff_cnt = 0  # for this section to observe, how many samples are effective with all_layers
window_eff_cnt = 0      # for this section to observe, how many samples are effective with any window 

for i, d in enumerate(good_samples):
#     for sect_k, sect_d in d['trace_scores'].items():
    sect_k = ob_sect_k
    sect_d = d['trace_scores'][sect_k]
    if sect_d['all_layers'] > 0.5:
        # not effective
        continue
    else:
        all_layers_eff_cnt += 1
        
    if min(sect_d['window'].values()) > 0.5:
        # not effective
        continue
    else:
        window_eff_cnt += 1
        
    window_d = sect_d['window']
    for l in range(1, 24):
        if window_d[str(l-1)] - window_d[str(l)] > _th:
            _info_d = {
                'id': i,
                'sect_k': sect_k,
                'layer': l,
                'last_layer_score': window_d[str(l-1)],
                'this_layer_score': window_d[str(l)],
            }
            check_info_l.append(_info_d)
            break
len(check_info_l), window_eff_cnt, all_layers_eff_cnt, len(good_samples)

(840, 916, 1207, 1207)

In [745]:
ctg_list = [(tm, nl) for tm in ['exact', 'partial', 'no-match'] for nl in ['1', '2', '3', '4+']]
layer_list = [str(l) for l in range(1, 24)]

ctg_elem2id = {elem : i for i, elem in enumerate(ctg_list)}
layer_elem2id = {elem : i for i, elem in enumerate(layer_list)}

cnt_matrix = np.zeros((len(ctg_list), len(layer_list)), int)

for info_d in check_info_l:
    sample_id = info_d['id']
    d = good_samples[sample_id]
    text_match = d['category']['text_match']
    node_len = d['category']['node_len']
    _ctg = (text_match, node_len)
    _layer = str(info_d['layer'])
    
    _ctg_idx = ctg_elem2id[_ctg]
    _layer_idx = layer_elem2id[_layer]
    cnt_matrix[_ctg_idx, _layer_idx] += 1

In [ ]:
# Create a figure and axis
fig, ax = plt.subplots(figsize=(8, 6))

# Display the matrix using imshow
im = ax.imshow(cnt_matrix, cmap='Blues')

# Set the tick labels for the first and second dimensions
ax.set_xticks(np.arange(len(layer_list)))
ax.set_yticks(np.arange(len(ctg_list)))

# Set the tick labels using the ctg_list and layer_list
ax.set_xticklabels(layer_list)
ax.set_yticklabels(ctg_list)

ax.set_title(f'Section: {ob_sect_k}\n')

# Rotate the x-axis tick labels if needed
# plt.xticks(rotation=90)

# Add a colorbar
cbar = ax.figure.colorbar(im, ax=ax, shrink=0.5)

# Show the plot
plt.show()


In [ ]:
# Create a figure and axis
fig, ax = plt.subplots(figsize=(8, 6))

# Display the matrix using imshow
im = ax.imshow(cnt_matrix, cmap='Blues')

# Set the tick labels for the first and second dimensions
ax.set_xticks(np.arange(len(layer_list)))
ax.set_yticks(np.arange(len(ctg_list)))

# Set the tick labels using the ctg_list and layer_list
ax.set_xticklabels(layer_list)
ax.set_yticklabels(ctg_list)

ax.set_title(f'Section: {ob_sect_k}\n')

# Rotate the x-axis tick labels if needed
# plt.xticks(rotation=90)

# Add a colorbar
cbar = ax.figure.colorbar(im, ax=ax, shrink=0.5)

# Show the plot
plt.show()


In [747]:
plt.close()

### Exp-5.3: attention section mutual removal

#### Load & Check

In [837]:
expect_type = 'table_alias'

res_path = f'/home/yshao/Projects/rome/results/exp5_3_attention_section_mutual_removal/exp=5.3.1_dev_{expect_type}_encoder-attn=self_attn-corrupt=zero.jsonl'

with open(res_path, 'r') as f:
    all_samples = [json.loads(l) for l in f]
len(all_samples)

1034

In [838]:
total_samples = 0
n_good_samples = 0
n_too_hard = 0      # wrong answer 
n_too_easy = 0      # base - low < 0.5

In [839]:
good_samples = []
bad_samples = []

for i, ex in enumerate(all_samples):
    for d in ex['trace_results']:
        total_samples += 1

        if d['is_good_sample']:
            n_good_samples += 1
            d['ex_id'] = i
            good_samples.append(d)
        elif not d['correct_prediction']:
            n_too_hard += 1
            bad_samples.append(d)
        else:
            assert d['base_score'] - d['low_score'] < 0.5
            n_too_easy += 1
            bad_samples.append(d)
            
total_samples, (n_good_samples, len(good_samples)), n_too_hard, n_too_easy, len(bad_samples), \
f'good / correct = {n_good_samples} / {n_good_samples + n_too_easy}'

(2039, (364, 364), 339, 1336, 1675, 'good / correct = 364 / 1700')

#### Overall avg

In [840]:
trace_scores_avg = {sect_k : defaultdict(int) for sect_k in good_samples[0]['trace_scores'].keys()}

for d in good_samples:
    for sect_k, sect_d in d['trace_scores'].items():
        for k, v in sect_d.items():
            if k == 'window':
                for l, s in v.items():
                    trace_scores_avg[sect_k][f'{k}-{l}'] += s
            else:
                s = v
                trace_scores_avg[sect_k][k] += s

for sect_k, sect_d in trace_scores_avg.items():
    for k, s in sect_d.items():
        sect_d[k] = s / len(good_samples)

In [ ]:
trace_scores_avg

#### Avg by aspects (category)

In [842]:
d['category']

{'sql_hardness': 'medium', 'node_role': 'where', 'text_match': 'no-match'}

In [843]:
# TEMP patch for node_len category 
for d in good_samples + bad_samples:
    node_len = len(d['answers_t'])
    assert len(mt_uskg.tokenizer.tokenize(d['expect'])) == node_len, (d['expect'], node_len)
    d['category']['node_len'] = str(node_len) if node_len <= 3 else '4+'

In [844]:
d['category']

{'sql_hardness': 'medium',
 'node_role': 'group by',
 'text_match': 'exact',
 'node_len': '3'}

In [845]:
# Key: (sect_k, aspect, asp_val, layer) -> [scores]
trace_scores_by_aspect = defaultdict(lambda: defaultdict(lambda: defaultdict(lambda: defaultdict(list))))
trace_scores_avg_by_aspect = defaultdict(lambda: defaultdict(lambda: defaultdict(lambda: defaultdict(float))))
trace_scores_cnt_by_aspect = defaultdict(lambda: defaultdict(int))  # no sect key & layer key 

In [846]:
for d in good_samples:
    for sect_k, sect_d in d['trace_scores'].items():
        for aspect, asp_val in d['category'].items():
            for k, v in sect_d.items():
                if k == 'window':
                    for l, s in v.items():
                        if not (int(l) % 4 == 3): continue
                        layer_k = f'{k}-{l}'
                        trace_scores_by_aspect[sect_k][aspect][asp_val][layer_k].append(s)
                else:
                    layer_k = k
                    s = v
                    trace_scores_by_aspect[sect_k][aspect][asp_val][layer_k].append(s)
                    
for sect_k, d1 in trace_scores_by_aspect.items():
    for asp_k, d2 in d1.items():
        for asp_v, d3 in d2.items():
            for layer_k, s in d3.items():
                trace_scores_avg_by_aspect[sect_k][asp_k][asp_v][layer_k] = np.mean(s)
                trace_scores_cnt_by_aspect[asp_k][asp_v] = len(s)

In [847]:
for sect_k, sect_d in trace_scores_avg_by_aspect.items():
    sect_d['overall'] = dict()
    for layer_k, s in trace_scores_avg[sect_k].items():
        if layer_k.startswith('window'):
            # only keep a subset of layers 
            _, l = layer_k.split('-')
            if not (int(l) % 4 == 3): continue
        sect_d['overall'][layer_k] = s

In [848]:
trace_scores_cnt_by_aspect

defaultdict(<function __main__.<lambda>()>,
            {'sql_hardness': defaultdict(int,
                         {'medium': 123, 'extra': 175, 'hard': 64, 'easy': 2}),
             'node_role': defaultdict(int,
                         {'select': 191,
                          'group by': 33,
                          'join': 12,
                          'where': 111,
                          'order by': 17}),
             'text_match': defaultdict(int,
                         {'exact': 230, 'partial': 28, 'no-match': 106}),
             'node_len': defaultdict(int, {'3': 364})})

In [ ]:
trace_scores_avg_by_aspect['c->p']

In [850]:
dump_d = ctu.nested_json_processing(trace_scores_avg_by_aspect, func=lambda x: np.format_float_positional(x, precision=4, min_digits=4))
# dump_d

In [851]:
dump_path = f'/home/yshao/Projects/rome/results/exp5_3_attention_section_mutual_removal/summ-exp=5.3.1_dev_{expect_type}_encoder-attn=self_attn-corrupt=zero.jsonl'

with open(dump_path, 'w') as f:
    json.dump(dump_d, f, indent=1)

#### (one-time temp patch)

In [804]:
# expect_type = 'table_alias'
# orig_res_path = f'/home/yshao/Projects/rome/results/exp5_3_attention_section_mutual_removal/no_c2p_exp=5.3.1_dev_{expect_type}_encoder-attn=self_attn-corrupt=zero.jsonl'
# add_res_path = f'/home/yshao/Projects/rome/results/exp5_3_attention_section_mutual_removal/exp=5.3.1+c2p_dev_{expect_type}_encoder-attn=self_attn-corrupt=zero.jsonl'

# merge_res_path = f'/home/yshao/Projects/rome/results/exp5_3_attention_section_mutual_removal/exp=5.3.1_dev_{expect_type}_encoder-attn=self_attn-corrupt=zero.jsonl'

In [805]:
# with open(orig_res_path, 'r') as f:
#     orig_all_samples = [json.loads(l) for l in f]
# with open(add_res_path, 'r') as f:
#     add_all_samples = [json.loads(l) for l in f]

# f = open(merge_res_path, 'w')
    
# for i, (orig_ex, add_ex) in enumerate(zip(orig_all_samples, add_all_samples)):
#     assert len(orig_ex['trace_results']) == len(add_ex['trace_results']), i
#     # There is randomness in the order of expected node (from set()), thus sorting here 
#     orig_ex['trace_results'].sort(key=lambda d: len(d['dec_prompt']))
#     add_ex['trace_results'].sort(key=lambda d: len(d['dec_prompt']))
#     for j, (orig_d, add_d) in enumerate(zip(orig_ex['trace_results'], add_ex['trace_results'])):
#         assert orig_d['is_good_sample'] == add_d['is_good_sample'], (i, j)
#         if not orig_d['is_good_sample']:
#             continue
            
#         # is good sample: add the new sections 
#         orig_d['trace_scores']['c->p'] = add_d['trace_scores']['c->p']
        
#         # put all at end in the dict 
#         _t = orig_d['trace_scores']['all']
#         del orig_d['trace_scores']['all']
#         orig_d['trace_scores']['all'] = _t
        
#     f.write(json.dumps(orig_ex, indent=None) + '\n')
    
# f.close()

## USKG error analysis

In [335]:
# adapted from play_pred()

def pred_sql(mt, ex):
    text_in = ex['text_in']
    struct_in = ex['struct_in']

    txt = f"{text_in}; structed knowledge: {struct_in}"
    
    tokenized_txt = mt.tokenizer_uskg([txt], max_length=1024, padding="max_length", truncation=True)
    
    device = mt.model.device
    pred = mt.tokenizer_uskg.batch_decode(
      mt.model.generate(
        torch.tensor(tokenized_txt.data['input_ids'], dtype=int, device=device),
        torch.tensor(tokenized_txt.data['attention_mask'], dtype=int, device=device),
        num_beams=1, 
        max_length=256
        ), 
      skip_special_tokens=True 
    )
    return pred[0]


In [336]:
# adapted from evaluator.evaluate_one()

def evaluate_sql(evaluator, db_name, gold, predicted):
    schema = evaluator.schemas[db_name]
    g_sql = sp_eval.get_sql(schema, gold)
    hardness = evaluator.eval_hardness(g_sql)
    # self.scores[hardness]["count"] += 1
    # self.scores["all"]["count"] += 1

    parse_error = False
    try:
        p_sql = sp_eval.get_sql(schema, predicted)
    except:
        # If p_sql is not valid, then we will use an empty sql to evaluate with the correct sql
        p_sql = {
            "except": None,
            "from": {"conds": [], "table_units": []},
            "groupBy": [],
            "having": [],
            "intersect": None,
            "limit": None,
            "orderBy": [],
            "select": [False, []],
            "union": None,
            "where": [],
        }

        # TODO fix
        parse_error = True

    # rebuild sql for value evaluation
    kmap = evaluator.kmaps[db_name]
    g_valid_col_units = sp_eval.build_valid_col_units(g_sql["from"]["table_units"], schema)
    g_sql = sp_eval.rebuild_sql_val(g_sql)
    g_sql = sp_eval.rebuild_sql_col(g_valid_col_units, g_sql, kmap)
    p_valid_col_units = sp_eval.build_valid_col_units(p_sql["from"]["table_units"], schema)
    p_sql = sp_eval.rebuild_sql_val(p_sql)
    p_sql = sp_eval.rebuild_sql_col(p_valid_col_units, p_sql, kmap)
    
    exec_score = None
    partial_scores = None
    exact_score = None
    if evaluator.etype in ["all", "exec"]:
        exec_score = sp_eval.eval_exec_match(
            evaluator.db_paths[db_name], predicted, gold, p_sql, g_sql
        )
        exec_score = int(exec_score)
    if evaluator.etype in ["all", "match"]:
        partial_scores = evaluator.eval_partial_match(p_sql, g_sql)
        exact_score = evaluator.eval_exact_match(p_sql, g_sql, partial_scores)
        # update_scores_match(self.scores, exact_score, hardness, partial_scores, PARTIAL_TYPES)

    return {
        "predicted": predicted,
        "gold": gold,
        "predicted_parse_error": parse_error,
        "hardness": hardness,
        "exact": exact_score,
        "partial": partial_scores,
        "exec": exec_score,
    }

In [354]:
db_dir = '/home/yshao/Projects/language/language/xsp/data/spider/database'

def execute_sql(db, sql_str):
    db_path = os.path.join(db_dir, db, f'{db}.sqlite')
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()
    try:
        cursor.execute(sql_str)
        res = cursor.fetchall()
    except:
        res = 'ERROR'
    conn.close()
    return res

In [403]:
ex = processed_spider_dev[503]
ex.keys()

dict_keys(['query', 'question', 'db_id', 'db_path', 'db_table_names', 'db_column_names', 'db_column_types', 'db_primary_keys', 'db_foreign_keys', 'rat_sql_graph', 'serialized_schema', 'struct_in', 'text_in', 'seq_out'])

In [404]:
pred = pred_sql(mt_uskg, ex)
ex['db_id'], ex['text_in'], ex['seq_out'], pred

('battle_death',
 "How many battles did not lose any ship with tonnage '225'?",
 "select count(*) from battle where id not in ( select lost_in_battle from ship where tonnage = '225' );",
 'select count(*) from battle where id not in ( select lost_in_battle from ship where tonnage > 225 )')

In [ ]:
text_in = ex['text_in']
struct_in = ex['struct_in']
txt = f"{text_in}; structed knowledge: {struct_in}"
txt_toks = mt_uskg.tokenizer_uskg.tokenize(txt)
len(txt_toks)

In [ ]:
txt_toks

In [405]:
exec_res = execute_sql(ex['db_id'], ex['seq_out'])
ex['seq_out'], exec_res

("select count(*) from battle where id not in ( select lost_in_battle from ship where tonnage = '225' );",
 [(7,)])

In [406]:
exec_res = execute_sql(ex['db_id'], pred)
pred, exec_res

('select count(*) from battle where id not in ( select lost_in_battle from ship where tonnage > 225 )',
 [(3,)])

In [407]:
chatgpt_pred = """
SELECT COUNT(DISTINCT battle.id) AS num_battles
FROM battle
LEFT JOIN ship ON battle.id = ship.lost_in_battle
WHERE (ship.tonnage != '225' OR ship.tonnage IS NULL)
"""

execute_sql(ex['db_id'], chatgpt_pred)

[(8,)]

In [339]:
eval_res = evaluate_sql(evaluator, db_name=spider_ex['db_id'], gold=spider_ex['seq_out'], predicted=pred)

In [ ]:
eval_res

In [411]:
# Eval all 
# TODO: rerun with tokenizer_uskg decoding 

eval_sql_results = []

for ex_id, ex in enumerate(tqdm(processed_spider_dev)):
    pred = pred_sql(mt_uskg, ex)
    eval_res = evaluate_sql(evaluator, db_name=ex['db_id'], gold=ex['seq_out'], predicted=pred)
    eval_sql_results.append(eval_res)
    

  0%|          | 0/1034 [00:00<?, ?it/s]

In [412]:
avg_exact = np.mean([d['exact'] for d in eval_sql_results])
avg_exec = np.mean([d['exec'] for d in eval_sql_results])
avg_exact, avg_exec

(0.6692456479690522, 0.6808510638297872)

In [ ]:
for i, d in enumerate(eval_sql_results):
    if d['exact'] and d['exec']:
        continue
    err_msg = ('A' if not d['exact'] else '') + ('X' if not d['exec'] else '')
    ex = processed_spider_dev[i]
    print(f'ID = {i}: {err_msg}  ({ex["db_id"]}) {ex["text_in"]}')
    print(f'Pred: {d["predicted"]}')
    print(f'Gold: {d["gold"]}')
    print()

In [ ]:
evaluator.schemas['dog_kennels'].schema

In [414]:
# acc by db_id 
eval_sql_results_by_db_id = defaultdict(list)

for i, d in enumerate(eval_sql_results):
    d['ex_id'] = i
    ex = processed_spider_dev[i]
    db_id = ex['db_id']
    eval_sql_results_by_db_id[db_id].append(d)

len(eval_sql_results_by_db_id)

20

In [415]:
for db_id, results in eval_sql_results_by_db_id.items():
    _avg_exact = np.mean([d['exact'] for d in results])
    _avg_exec = np.mean([d['exec'] for d in results])
    print(f'{db_id}\t{_avg_exact:.4f}\t{_avg_exec:.4f}')

concert_singer	0.8889	0.8889
pets_1	0.5714	0.7381
car_1	0.3478	0.3913
flight_2	0.7000	0.7500
employee_hire_evaluation	0.9474	0.9737
cre_Doc_Template_Mgt	0.8333	0.9048
course_teach	0.8667	0.9333
museum_visit	0.7222	0.8333
wta_1	0.6774	0.6129
battle_death	0.5000	0.5000
student_transcripts_tracking	0.6667	0.6795
tvshow	0.7258	0.6613
poker_player	0.8750	0.8750
voter_1	0.6000	0.6667
world_1	0.5083	0.4833
orchestra	0.8000	0.8750
network_1	0.6250	0.4643
dog_kennels	0.5854	0.5976
singer	0.8667	0.8667
real_estate_properties	0.5000	0.5000


## Model intermediate inspect

In [ ]:
[k for k, v in mt_uskg.model.named_parameters()]

### Self-attention

In [284]:
_t5_config = mt_uskg.model.config
type(_t5_config)

transformers.models.t5.configuration_t5.T5Config

In [ ]:
_t5_config

In [306]:
sa_module = uskg.models.prompt.modeling_t5.T5Attention(_t5_config)
sa_module.eval()
sa_module

T5Attention(
  (q): Linear(in_features=1024, out_features=1024, bias=False)
  (k): Linear(in_features=1024, out_features=1024, bias=False)
  (v): Linear(in_features=1024, out_features=1024, bias=False)
  (o): Linear(in_features=1024, out_features=1024, bias=False)
)

In [ ]:
sa_module.__dict__

In [308]:
bsz = 3
seq_len = 10
dim = 1024
fake_sa_input = torch.zeros(bsz, seq_len, dim)

In [309]:
attn_output, present_key_value_state, position_bias, attn_weights = sa_module(fake_sa_input, output_attentions=True)

In [310]:
attn_output.size()

torch.Size([3, 10, 1024])

In [311]:
present_key_value_state

In [312]:
attn_weights.size()

torch.Size([3, 16, 10, 10])

In [313]:
position_bias.size()

torch.Size([1, 16, 10, 10])

In [ ]:
attn_weights

## Tests

### create_analysis_samples

In [862]:
ex_id = 111
a_ex_id = 0

ex = processed_spider_dev[ex_id]
ex['text_in'], \
ex['struct_in'], \
ex['seq_out']

('What is the accelerate of the car make amc hornet sportabout (sw)?',
 '| car_1 | continents : contid , continent | countries : countryid , countryname , continent | car_makers : id , maker ( amc ) , fullname , country | model_list : modelid , maker , model ( amc ) | car_names : makeid , model ( amc ) , make ( amc hornet , amc hornet sportabout (sw) ) | cars_data : id , mpg , cylinders , edispl , horsepower , weight , accelerate , year',
 "select t1.accelerate from cars_data as t1 join car_names as t2 on t1.id = t2.makeid where t2.make = 'amc hornet sportabout (sw)';")

In [863]:
# temp test
# ex['seq_out'] = 'select year from cars_data'

In [871]:
a_ex_list = ctu.create_analysis_sample_dicts(
                mt_uskg, ex,
                subject_type='column',
                remove_struct_duplicate_nodes=True)

In [872]:
a_ex_list[a_ex_id].keys()

dict_keys(['query', 'question', 'db_id', 'db_path', 'db_table_names', 'db_column_names', 'db_column_types', 'db_primary_keys', 'db_foreign_keys', 'rat_sql_graph', 'serialized_schema', 'struct_in', 'text_in', 'seq_out', 'enc_sentence', 'enc_tokenized', 'text_range', 'struct_range', 'struct_node_ranges_dict', 'dec_prompt', 'expect', 'expect_type', 'remove_struct_duplicate_nodes', 'parsed_struct_in', 'col2table', 'token_ranges_dict', 'node_name_ranges', 'expect_input_ranges', 'alias2table', 'self_ranges', 'context_ranges', 'category'])

In [873]:
a_ex_list[a_ex_id]['alias2table']

{'t1': 'cars_data', 't2': 'car_names'}

In [ ]:
[(d['dec_prompt'], d['expect'], d['node_name_ranges'], d['expect_input_ranges'], '------',\
  d['self_ranges'], d['context_ranges'],\
  d['category'], '------' * 2) for d in a_ex_list]

In [ ]:
d = dict(a_ex_list[a_ex_id])
d

In [ ]:
d = ctu.add_clean_prediction(mt_uskg, d)

In [ ]:
d

#### parse_sql_alias2table

In [185]:
_sql = 'SELECT t2.aaa , t3.ccc FROM table_name as t1 JOIN other_table as t2 on table_name.a_a = other_table.b_a JOIN ttt as t3 on other_table.asth = ttt.asth'
ctu.parse_sql_alias2table(_sql)

{'t1': 'table_name', 't2': 'other_table', 't3': 'ttt'}

#### for syntax

In [861]:
_sql = 'SELECT t2.aaa, DISTINCT(t3.ccc), COUNT(*) FROM table_name as t1 JOIN other_table as t2 on table_name.a_a = other_table.b_a JOIN ttt as t3 on other_table.asth = ttt.asth WHERE t2.col like %hey%'.lower()
prompts = ctu.make_syntax_dec_prompt(_sql, '%', is_punct=True)
prompts

['select t2.aaa, distinct(t3.ccc), count(*) from table_name as t1 join other_table as t2 on table_name.a_a = other_table.b_a join ttt as t3 on other_table.asth = ttt.asth where t2.col like',
 'select t2.aaa, distinct(t3.ccc), count(*) from table_name as t1 join other_table as t2 on table_name.a_a = other_table.b_a join ttt as t3 on other_table.asth = ttt.asth where t2.col like %hey']

In [875]:
a_ex_list_syntax = ctu.create_syntax_analysis_sample_dicts(
                mt_uskg, ex)

AssertionError: ('.', ['select', 't1', '.', 'accelerate', 'from', 'cars_data', 'as', 't1', 'join', 'car_names', 'as', 't2', 'on', 't1', '.', 'id', '=', 't2', '.', 'makeid', 'where', 't2', '.', 'make', '=', "'", 'amc', 'hornet', 'sportabout', '(', 'sw', ')', "'", ';'])

### context_ranges

In [249]:
a_ex = dict(a_ex_list[0])
a_ex['text_in'], a_ex['struct_in'], a_ex['seq_out'], a_ex['expect']

('What is the name of the different car makers who produced a car in 1970?',
 '| car_1 | continents : contid , continent | countries : countryid , countryname , continent | car_makers : id , maker , fullname , country | model_list : modelid , maker , model | car_names : makeid , model , make | cars_data : id , mpg , cylinders , edispl , horsepower , weight , accelerate , year',
 'select year from cars_data',
 'year')

In [250]:
a_ex['alias2table']

{}

In [256]:
text_range = a_ex['text_range']
struct_range = a_ex['struct_range']

# For full context tokens, use [0, L] and [R, -1]
# L: node left max end index ; R: node right min start index

token_ranges_dict = a_ex['token_ranges_dict']
_all_node_range_lists = list(token_ranges_dict['col_name_ranges'].values()) + list(token_ranges_dict['table_name_ranges'].values()) + list(token_ranges_dict['db_id_ranges'].values())
_all_node_ranges = [rg
                    for rg_list in _all_node_range_lists
                    for rg in rg_list]
_all_left_endpoint = [s for s, e in _all_node_ranges] + [struct_range[1]]
_all_right_endpoint = [e for s, e in _all_node_ranges] + [struct_range[0]]
# TODO: pull this part out to the shared function (e.g. create_analysis_sample_dicts)
# TODO: test for columns on ends

expect_input_ranges = a_ex['expect_input_ranges']    # list of ranges of node-of-interest (code allows dup)
# tok_indices = [i for s, e in expect_input_ranges for i in range(s, e)]
# expect_input_indices = [i for s, e in expect_input_ranges for i in range(s, e)]
enc_sentence = a_ex['enc_sentence']
dec_prompt = a_ex['dec_prompt']
# node = a_ex['expect']

context_range_endpoints = [struct_range[0]]
self_range_endpoints = []       # This is different from `expect_input_ranges`: this includes boundary toks
for tok_s, tok_e in expect_input_ranges:
    _l = max([e for e in _all_right_endpoint if e <= tok_s])
    _r = min([s for s in _all_left_endpoint if s >= tok_e])
    context_range_endpoints.extend([_l, _r])
    self_range_endpoints.extend([_l, _r])
context_range_endpoints.append(struct_range[1])

self_ranges = [(self_range_endpoints[i], self_range_endpoints[i+1])
                for i in range(0, len(self_range_endpoints), 2)]
self_tok_indices = [i for s, e in self_ranges for i in range(s, e)]

context_ranges = [(context_range_endpoints[i], context_range_endpoints[i+1])
                    for i in range(0, len(context_range_endpoints), 2)]
context_ranges = [(s, e) for s, e in context_ranges if e > s]
context_tok_indices = corrupt_tok_indices = [i for s, e in context_ranges for i in range(s, e)]

text_tok_indices = list(range(*text_range))


In [257]:
self_ranges, context_ranges

([(137, 140)], [(24, 137)])

In [ ]:
context_tok_indices

In [265]:
for s, e in self_ranges:
    _piece = tokenizer.decode(a_ex['enc_tokenized']['input_ids'][s : e])
    print(_piece)

, year


In [254]:
for s, e in context_ranges:
    _piece = tokenizer.decode(a_ex['enc_tokenized']['input_ids'][s : e])
    print(_piece)

| car_1 | continents : contid, continent | countries : countryid, countryname, continent | car_makers : id, maker, fullname, country | model_list : modelid, maker, model | car_names : makeid, model, make | cars_data : id, mpg, cylinders, edispl, horsepower, weight, accelerate



### run_uskg_forward

In [89]:
a_ex = dict(a_ex_list[a_ex_id])

In [90]:
enc_sentence = a_ex['enc_sentence']
dec_prompt = a_ex['dec_prompt']
expect = a_ex['expect']

In [91]:
enc_sentence, dec_prompt, expect

('What is the name of the different car makers who produced a car in 1970?; structed knowledge: | car_1 | continents : contid , continent | countries : countryid , countryname , continent | car_makers : id , maker , fullname , country | model_list : modelid , maker , model | car_names : makeid , model , make | cars_data : id , mpg , cylinders , edispl , horsepower , weight , accelerate , year',
 'select distinct t1.maker from car_makers as t1 join model_list as t2 on t1.id = t2.maker join car_names as t3 on t2.model = t3.model join cars_data as t4 on t3.',
 'makeid')

In [318]:
inp = ctu.make_inputs_t5(
    mt_uskg.tokenizer,
    [enc_sentence] * 11,
    [dec_prompt] * 11,
    answer=expect)

In [320]:
_out = ctu.run_model_forward_uskg(mt_uskg.model, **inp, output_attentions=True)

In [322]:
_out.__dict__.keys()

dict_keys(['loss', 'logits', 'past_key_values', 'decoder_hidden_states', 'decoder_attentions', 'cross_attentions', 'encoder_last_hidden_state', 'encoder_hidden_states', 'encoder_attentions'])

In [329]:
# layers, (bsz, n_heads, seq_len, seq_len + prev_len)
len(_out.encoder_attentions), _out.encoder_attentions[0].size()

(24, torch.Size([11, 16, 141, 151]))

In [330]:
# layers, (bsz, n_heads, prompt_len, seq_len + prev_len)
len(_out.cross_attentions), _out.cross_attentions[0].size()

(24, torch.Size([11, 16, 2, 151]))

In [331]:
# layers, (bsz, n_heads, prompt_len, prompt_len + prev_len)
len(_out.decoder_attentions), _out.decoder_attentions[0].size()

(24, torch.Size([11, 16, 2, 12]))

In [339]:
inp.keys()

dict_keys(['input_ids', 'attention_mask', 'decoder_input_ids', 'decoder_attention_mask'])

### trace

In [207]:
a_ex = dict(a_ex_list[a_ex_id])
a_ex = ctu.add_clean_prediction(mt_uskg, a_ex)

In [208]:
result = ctu.make_basic_result_dict(a_ex)
result

{'enc_sentence': 'Which city has the most frequent destination airport?; structed knowledge: | flight_2 | airlines : uid , airline , abbreviation , country | airports : city , airportcode , airportname , country , countryabbrev | flights : airline , flightno , sourceairport , destairport',
 'seq_out': 'select t1.city from airports as t1 join flights as t2 on t1.airportcode = t2.destairport group by t1.city order by count(*) desc limit 1',
 'dec_prompt': 'select t1.city from airports as t1 join flights as t2 on t1.airportcode = t2.destairport group by t1.',
 'expect': 'city',
 'expect_type': 'column',
 'db_id': 'flight_2',
 'expect_input_ranges': [(45, 46)],
 'expect_table': 'airports',
 'answer': 'city',
 'base_score': 0.9983423948287964,
 'answers_t': [6726],
 'correct_prediction': True,
 'category': {'sql_hardness': 'extra',
  'node_role': 'group by',
  'text_match': 'exact'}}

In [209]:
enc_sentence = a_ex['enc_sentence']
dec_prompt = a_ex['dec_prompt']
expect = a_ex['expect']
answer = result['answer']
answers_t = result['answers_t']

inp = ctu.make_inputs_t5(
    mt_uskg.tokenizer,
    [enc_sentence] * 11,
    [dec_prompt] * 11,
    answer=expect)

text_range = a_ex['text_range']
struct_range = a_ex['struct_range']

self_ranges = a_ex['self_ranges']
context_ranges = a_ex['context_ranges']

self_tok_indices = [i for s, e in self_ranges for i in range(s, e)]
context_tok_indices = corrupt_tok_indices = [i for s, e in context_ranges for i in range(s, e)]
text_tok_indices = list(range(*text_range))
struct_tok_indices = list(range(*struct_range))

In [210]:
_score = ctu.trace_with_repatch_uskg(
    model=mt_uskg.model,
    inp=inp,
#     states_to_patch=[(tnum, ctu.layername_uskg(mt_uskg.model, 'encoder', mt_uskg.num_enc_layers - 1))
#                     for tnum in range(*struct_range)],
    states_to_patch=[],
    states_to_unpatch=[],
    answers_t=answers_t,
    tokens_to_mix=text_tok_indices,
    tokens_to_mix_individual_indices=True,
    replace=True,
).item()

In [211]:
answers_t, answer, _score

([6726], 'city', 0.8450507521629333)

In [212]:
states_to_corrupt = [(tnum, ctu.layername_uskg(mt_uskg.model, "encoder", 0, "embed"))
                for tnum in text_tok_indices]

_score = ctu.trace_with_repatch_uskg(
    model=mt_uskg.model,
    inp=inp,
    states_to_patch=[],
    states_to_unpatch=[],
    answers_t=answers_t,
    states_to_corrupt=states_to_corrupt,
#     tokens_to_mix=corrupt_tok_indices,
#     tokens_to_mix_individual_indices=True,
    replace=True,
).item()
_score

0.8450507521629333

In [213]:
# Pair of identical input to test correctness 

_score = ctu.trace_with_repatch_uskg(
    model=mt_uskg.model,
    inp=inp,
#     states_to_patch=[(tnum, ctu.layername_uskg(mt_uskg.model, 'encoder', l))
#                     for tnum in text_tok_indices for l in range(mt_uskg.num_enc_layers - 1)],
    states_to_patch=[],
    states_to_patch_1st_pass=[(tnum, ctu.layername_uskg(mt_uskg.model, 'encoder', 12))
                    for tnum in text_tok_indices],
    states_to_unpatch=[(tnum, ctu.layername_uskg(mt_uskg.model, 'encoder', 23))
                    for tnum in struct_tok_indices],
    answers_t=answers_t,
    tokens_to_mix=text_tok_indices,
    tokens_to_mix_individual_indices=True,
    tokens_to_mix_1st_pass=context_tok_indices,
    replace=True,
).item()

_score

tensor(0.9952, device='cuda:0')

In [214]:
_score = ctu.trace_with_repatch_uskg(
    model=mt_uskg.model,
    inp=inp,
#     states_to_patch=[(tnum, ctu.layername_uskg(mt_uskg.model, 'encoder', l))
#                     for tnum in text_tok_indices for l in range(mt_uskg.num_enc_layers - 1)],
    states_to_patch=[],
    states_to_patch_1st_pass=[(tnum, ctu.layername_uskg(mt_uskg.model, 'encoder', 12))
                    for tnum in text_tok_indices],
    states_to_unpatch=[(tnum, ctu.layername_uskg(mt_uskg.model, 'encoder', 23))
                    for tnum in struct_tok_indices],
    answers_t=answers_t,
    states_to_corrupt=[(tnum, ctu.layername_uskg(mt_uskg.model, "encoder", 0, "embed"))
                    for tnum in text_tok_indices],
    states_to_corrupt_1st_pass=[(tnum, ctu.layername_uskg(mt_uskg.model, "encoder", 0, "embed"))
                    for tnum in context_tok_indices],
    replace=True,
).item()

_score

tensor(0.9952, device='cuda:0')

In [215]:
# Test corrupting attention 
_score = ctu.trace_with_repatch_uskg(
    model=mt_uskg.model,
    inp=inp,
#     states_to_patch=[(tnum, ctu.layername_uskg(mt_uskg.model, 'encoder', l))
#                     for tnum in text_tok_indices for l in range(mt_uskg.num_enc_layers - 1)],
    states_to_patch=[],
    states_to_unpatch=[],
    answers_t=answers_t,
    states_to_corrupt=[(tnum, ctu.layername_uskg(mt_uskg.model, "encoder", l, "self_attn"))
                    for tnum in text_tok_indices for l in range(mt_uskg.num_enc_layers)],
    replace=True,
).item()

_score

0.7253002524375916

In [ ]:
[n for n, w in mt_uskg.model.named_parameters()]

In [216]:
vocab_probs = ctu.run_repatch_uskg_multi_token(
    model=mt_uskg.model,
    inp=inp,
#     states_to_patch=[(tnum, ctu.layername_uskg(mt_uskg.model, 'encoder', mt_uskg.num_enc_layers - 1))
#                     for tnum in range(*struct_range)],
    states_to_patch=[(tnum, ctu.layername_uskg(mt_uskg.model, 'encoder', l))
                    for tnum in self_tok_indices for l in range(mt_uskg.num_enc_layers - 1)],
    states_to_unpatch=[(tnum, ctu.layername_uskg(mt_uskg.model, 'encoder', mt_uskg.num_enc_layers - 1))
                    for tnum in self_tok_indices],
    answer_len=len(answers_t),
    tokens_to_mix=corrupt_tok_indices,
    tokens_to_mix_individual_indices=True,
    replace=True,
)

In [217]:
vocab_probs.size()

torch.Size([1, 32102])

In [218]:
torch.max(vocab_probs, dim=-1)

torch.return_types.max(
values=tensor([1.], device='cuda:0'),
indices=tensor([7634], device='cuda:0'))

In [219]:
vocab_probs[0, 7634]

tensor(1., device='cuda:0')

In [220]:
vocab_probs

tensor([[2.2642e-25, 1.2223e-15, 7.3942e-18,  ..., 9.1578e-20, 2.6884e-39,
         2.8131e-39]], device='cuda:0')

### trace - partial edit

#### single attention layer

In [385]:
from uskg.models.prompt.modeling_t5 import T5Attention

In [ ]:
_t5_config = copy.deepcopy(mt_uskg.model.config)
_t5_config.d_model = 10
_t5_config.d_kv = 4
_t5_config.num_heads = 3
_t5_config

In [387]:
test_attn_module = T5Attention(config=_t5_config)
test_attn_module.eval()
test_attn_module

T5Attention(
  (q): Linear(in_features=10, out_features=12, bias=False)
  (k): Linear(in_features=10, out_features=12, bias=False)
  (v): Linear(in_features=10, out_features=12, bias=False)
  (o): Linear(in_features=12, out_features=10, bias=False)
)

In [388]:
[(k, v.size()) for k, v in test_attn_module.named_parameters()]

[('q.weight', torch.Size([12, 10])),
 ('k.weight', torch.Size([12, 10])),
 ('v.weight', torch.Size([12, 10])),
 ('o.weight', torch.Size([10, 12]))]

In [389]:
[k for k, v in test_attn_module.named_modules()]

['', 'q', 'k', 'v', 'o']

In [426]:
_test_bs = 1
_test_seqlen = 7
_test_prevlen = 2

_test_h = torch.randn(_test_bs, _test_seqlen, _t5_config.d_model)
# _test_h[:, 0] = 999.0
_test_prefix = {
    'prev_key': torch.randn(_test_bs, _t5_config.num_heads, _test_prevlen, _t5_config.d_kv),
    'prev_value': torch.randn(_test_bs, _t5_config.num_heads, _test_prevlen, _t5_config.d_kv),
    'prev_key_padding_mask': torch.zeros(_test_bs, _test_prevlen).bool()
}
_test_h.size(), [(k, v.size()) for k, v in _test_prefix.items()]

(torch.Size([1, 7, 10]),
 [('prev_key', torch.Size([1, 3, 2, 4])),
  ('prev_value', torch.Size([1, 3, 2, 4])),
  ('prev_key_padding_mask', torch.Size([1, 2]))])

In [427]:
_test_h

tensor([[[ 0.8561, -0.0552,  0.3845,  1.1441, -1.2765, -0.6869, -0.9646,
          -1.0255, -2.3659, -1.7153],
         [ 1.0685, -0.1617,  0.8310, -1.7257,  0.3674,  1.7559,  0.5763,
          -0.9344,  0.9016,  0.7490],
         [-1.1887, -1.0820, -0.5925,  0.7623, -0.6538, -0.0067,  0.5618,
           1.3310,  1.2580, -0.6973],
         [ 0.2807,  0.0763, -0.3539,  0.9494, -0.1557, -0.7645, -0.2103,
          -1.0175, -0.3029, -0.0376],
         [ 0.0984,  0.5610, -2.3323,  1.3421, -1.0381, -1.8568, -0.7754,
          -1.6037,  0.2501, -1.4155],
         [-1.4474, -0.4784,  0.0972, -0.3393,  1.2340,  0.7611, -0.4786,
           0.0506, -0.1188,  2.7051],
         [ 2.0642, -0.0186, -0.8283,  1.0852,  0.9819, -0.4044,  0.9831,
          -0.2723,  0.2037,  1.6401]]])

In [428]:
_test_prefix

{'prev_key': tensor([[[[-2.2578,  0.2019,  0.3287,  0.3906],
           [-1.2143, -0.7039, -1.0298,  0.1425]],
 
          [[ 0.3967,  1.5168,  0.0967,  1.4454],
           [ 0.1648,  0.2483,  1.5992,  1.2469]],
 
          [[ 1.3875,  0.4460, -0.2676, -1.2290],
           [ 2.0209,  1.1736,  0.8446,  0.8827]]]]),
 'prev_value': tensor([[[[ 0.8351,  1.2052,  1.4187, -0.5358],
           [-1.4274,  0.2792,  2.0149,  1.3695]],
 
          [[-0.0379,  1.8999, -0.4236, -0.9176],
           [ 1.5794,  1.1735, -0.2925,  2.2855]],
 
          [[ 1.1935,  0.9343, -0.5582,  0.8163],
           [-1.3765,  0.4046,  1.0941,  0.4058]]]]),
 'prev_key_padding_mask': tensor([[False, False]])}

In [429]:
_test_mask = torch.zeros(1, 1, 1, _test_seqlen)

In [ ]:
_out = test_attn_module.forward(
    _test_h,
    mask=_test_mask,
    prefix=_test_prefix,
    output_attentions=True,
)

In [ ]:
_out

In [432]:
_out[3].sum(-1)

tensor([[[0.3544, 0.3264, 0.2939, 0.3878, 0.4695, 0.2997, 0.1775],
         [0.4028, 0.1655, 0.2949, 0.5148, 0.4162, 0.4189, 0.4683],
         [0.5839, 0.4639, 0.1393, 0.4925, 0.6272, 0.1917, 0.4634]]])

In [433]:
def _test_mask_attention(att, mix_mask):
    # token 4,5,6 (in real_seq) not attending to 2 (in full_seq, i.e. 0 in real_seq)
    # att: (bs, n_head, real_seq, full_seq)
#     att[:, :, 4:, 2] = 0.0

    _zero = torch.tensor(0, dtype=att.dtype)
    att = torch.where(mix_mask, _zero, att)

    print(att)
    return att

In [434]:
mix_mask = torch.zeros(1, 1, _test_seqlen, _test_seqlen + _test_prevlen, dtype=bool)
mix_mask[:, :, ::2, ::2] = 1

mix_mask_2 = torch.zeros(1, 1, _test_seqlen, _test_seqlen + _test_prevlen, dtype=bool)
mix_mask_2[:, :, :, :_test_prevlen] = 1
mix_mask_2[:, :, 3:, _test_prevlen : _test_prevlen+3] = 1
mix_mask_2[:, :, :3, _test_prevlen+3 :] = 1

mix_mask, mix_mask_2

(tensor([[[[ True, False,  True, False,  True, False,  True, False,  True],
           [False, False, False, False, False, False, False, False, False],
           [ True, False,  True, False,  True, False,  True, False,  True],
           [False, False, False, False, False, False, False, False, False],
           [ True, False,  True, False,  True, False,  True, False,  True],
           [False, False, False, False, False, False, False, False, False],
           [ True, False,  True, False,  True, False,  True, False,  True]]]]),
 tensor([[[[ True,  True, False, False, False,  True,  True,  True,  True],
           [ True,  True, False, False, False,  True,  True,  True,  True],
           [ True,  True, False, False, False,  True,  True,  True,  True],
           [ True,  True,  True,  True,  True, False, False, False, False],
           [ True,  True,  True,  True,  True, False, False, False, False],
           [ True,  True,  True,  True,  True, False, False, False, False],
        

In [ ]:
mix_mask | mix_mask_2

In [435]:
test_attn_module.ext_attention_weights_fn = lambda att : _test_mask_attention(att, mix_mask_2)

In [436]:
print(test_attn_module.ext_attention_weights_fn)
print(test_attn_module.ext_attention_logits_fn)

<function <lambda> at 0x7fc33d6065e0>
None


In [ ]:
_out = test_attn_module.forward(
    _test_h,
    mask=_test_mask,
    prefix=_test_prefix,
    output_attentions=True,
)

In [ ]:
_out

#### full model

In [438]:
a_ex = dict(a_ex_list[a_ex_id])
a_ex = ctu.add_clean_prediction(mt_uskg, a_ex)

In [439]:
result = ctu.make_basic_result_dict(a_ex)
result

{'enc_sentence': 'What is the accelerate of the car make amc hornet sportabout (sw)?; structed knowledge: | car_1 | continents : contid , continent | countries : countryid , countryname , continent | car_makers : id , maker ( amc ) , fullname , country | model_list : modelid , maker , model ( amc ) | car_names : makeid , model ( amc ) , make ( amc hornet , amc hornet sportabout (sw) ) | cars_data : id , mpg , cylinders , edispl , horsepower , weight , accelerate , year',
 'seq_out': "select t1.accelerate from cars_data as t1 join car_names as t2 on t1.id = t2.makeid where t2.make = 'amc hornet sportabout (sw)';",
 'dec_prompt': 'select t1.accelerate from cars_data as t1 join car_names as t2 on t1.id = t2.makeid where t2.',
 'expect': 'make',
 'expect_type': 'column',
 'db_id': 'car_1',
 'expect_input_ranges': [(121, 145)],
 'expect_table': 'car_names',
 'answer': 'make',
 'base_score': 0.9998800754547119,
 'answers_t': [19509],
 'correct_prediction': True,
 'category': {'sql_hardness':

In [440]:
enc_sentence = a_ex['enc_sentence']
dec_prompt = a_ex['dec_prompt']
expect = a_ex['expect']
answer = result['answer']
answers_t = result['answers_t']

inp = ctu.make_inputs_t5(
    mt_uskg.tokenizer,
    [enc_sentence] * 2,
    [dec_prompt] * 2,
    answer=expect)

text_range = a_ex['text_range']
struct_range = a_ex['struct_range']

self_ranges = a_ex['self_ranges']
context_ranges = a_ex['context_ranges']

self_tok_indices = [i for s, e in self_ranges for i in range(s, e)]
context_tok_indices = corrupt_tok_indices = [i for s, e in context_ranges for i in range(s, e)]
text_tok_indices = list(range(*text_range))
struct_tok_indices = list(range(*struct_range))

In [441]:
_score = ctu.trace_with_repatch_uskg(
    model=mt_uskg.model,
    inp=inp,
#     states_to_patch=[(tnum, ctu.layername_uskg(mt_uskg.model, 'encoder', mt_uskg.num_enc_layers - 1))
#                     for tnum in range(*struct_range)],
    states_to_patch=[],
    states_to_unpatch=[],
    answers_t=answers_t,
    tokens_to_mix=context_tok_indices,
    tokens_to_mix_individual_indices=True,
    replace=True,
).item()

In [442]:
answers_t, answer, _score

([19509], 'make', 0.9987825751304626)

In [451]:
_score = ctu.trace_attention_manip_uskg_multi_token(
# _probs = ctu.run_attention_manip_uskg_multi_token(
    model=mt_uskg.model,
    inp=inp,
#     answer_len=len(answers_t),
    answers_t=answers_t,
#     states_to_patch=[],
    layers_to_mix=[ctu.layername_uskg(mt_uskg.model, 'encoder', l, 'self_attn') for l in range(24)],
    src_tokens_to_mix=text_tok_indices + struct_tok_indices, # src doesn't have prefix 
#     src_tokens_to_mix=[-1],
    tgt_tokens_to_mix=list(range(10)) + [i + 10 for i in text_tok_indices + struct_tok_indices],  # tgt has prefix 
)

In [452]:
_probs.max(dim=-1)
# _score

torch.return_types.max(
values=tensor([0.6447], device='cuda:0'),
indices=tensor([4350], device='cuda:0'))

In [453]:
mt_uskg.tokenizer.decode([4350])

'name'

In [110]:
# ctu.layername_uskg(mt_uskg.model, 'encoder', 0, 'self_attn')

### utils

#### nested_json_processing

In [120]:
a = np.random.randn(2,2,2,2).tolist()
a

[[[[0.4967141530112327, -0.13826430117118466],
   [0.6476885381006925, 1.5230298564080254]],
  [[-0.23415337472333597, -0.23413695694918055],
   [1.5792128155073915, 0.7674347291529088]]],
 [[[-0.4694743859349521, 0.5425600435859647],
   [-0.46341769281246226, -0.46572975357025687]],
  [[0.24196227156603412, -1.913280244657798],
   [-1.7249178325130328, -0.5622875292409727]]]]

In [124]:
# ctu.nested_list_processing(a, func=lambda x: np.format_float_positional(x, precision=2))

In [129]:
b = {
    'a': a,
    'a_list': {
        'a0': a[0],
        'a1_list': {
            'a10': a[1][0],
            'a11': a[1][1],
        },
    },
}

b

{'a': [[[[0.4967141530112327, -0.13826430117118466],
    [0.6476885381006925, 1.5230298564080254]],
   [[-0.23415337472333597, -0.23413695694918055],
    [1.5792128155073915, 0.7674347291529088]]],
  [[[-0.4694743859349521, 0.5425600435859647],
    [-0.46341769281246226, -0.46572975357025687]],
   [[0.24196227156603412, -1.913280244657798],
    [-1.7249178325130328, -0.5622875292409727]]]],
 'a_list': {'a0': [[[0.4967141530112327, -0.13826430117118466],
    [0.6476885381006925, 1.5230298564080254]],
   [[-0.23415337472333597, -0.23413695694918055],
    [1.5792128155073915, 0.7674347291529088]]],
  'a1_list': {'a10': [[-0.4694743859349521, 0.5425600435859647],
    [-0.46341769281246226, -0.46572975357025687]],
   'a11': [[0.24196227156603412, -1.913280244657798],
    [-1.7249178325130328, -0.5622875292409727]]}}}

In [131]:
ctu.nested_json_processing(b, func=lambda x: np.format_float_positional(x, precision=2))

{'a': [[[['0.5', '-0.14'], ['0.65', '1.52']],
   [['-0.23', '-0.23'], ['1.58', '0.77']]],
  [[['-0.47', '0.54'], ['-0.46', '-0.47']],
   [['0.24', '-1.91'], ['-1.72', '-0.56']]]],
 'a_list': {'a0': [[['0.5', '-0.14'], ['0.65', '1.52']],
   [['-0.23', '-0.23'], ['1.58', '0.77']]],
  'a1_list': {'a10': [['-0.47', '0.54'], ['-0.46', '-0.47']],
   'a11': [['0.24', '-1.91'], ['-1.72', '-0.56']]}}}

## Temp

### Debugging exp

In [497]:
ex = processed_spider_dev[97]
text_in = ex['text_in']
struct_in = ex['struct_in']

enc_sentence = f"{text_in}; structed knowledge: {struct_in}"
dec_prompt = "select t1.model from"
expect = "car_names"

In [498]:
inp = ctu.make_inputs_t5(
    mt_uskg.tokenizer,
    enc_sentences=[enc_sentence]*11,
    dec_prompts=[dec_prompt]*11,
    answer=expect
)

In [499]:
ans_toks = decode_tokens(mt_uskg.tokenizer, mt_uskg.tokenizer.encode(expect, add_special_tokens=False))
ans_toks

['car', '_', 'name', 's']

In [500]:
mt_uskg.tokenizer.decode(inp['decoder_input_ids'][0])

'<pad> select t1.model from car_name'

In [501]:
mt_uskg.tokenizer.tokenize(dec_prompt), len(mt_uskg.tokenizer.tokenize(dec_prompt))

(['▁select', '▁', 't', '1.', 'model', '▁from'], 6)

In [502]:
len(inp['input_ids'][0])

139

In [503]:
# r = ctu.trace_with_patch_uskg_multi_token(
#     mt_uskg.model,
#     inp=inp,
# #     states_to_patch=[(4, ctu.layername_uskg(mt_uskg.model, 'decoder', 3))],
#     states_to_patch=[],
# #     answers_t=mt_uskg.tokenizer.encode(expect, add_special_tokens=False),
# #     tokens_to_mix=(0, len(inp['input_ids'][0])-1),
#     tokens_to_mix=None,
#     replace=True,
# )

In [504]:
# r

[tensor(0.5643, device='cuda:0'),
 tensor(1.0000, device='cuda:0'),
 tensor(0.9996, device='cuda:0'),
 tensor(1.0000, device='cuda:0')]

In [506]:
answer_len = len(mt_uskg.tokenizer.tokenize(expect))
pred_out = ctu.predict_from_input_uskg_multi_token(mt_uskg.model, inp, pred_len=answer_len)
pred, p = pred_out
pred[0]

tensor([ 443,  834, 4350,    7], device='cuda:0')

In [224]:
# pred_toks = decode_tokens(mt_uskg.tokenizer, pred[0])
# list(zip(pred_toks, ans_toks))

[('model', 'car'), ('_', '_'), ('name', 'name'), ('s', 's')]

In [507]:
ex['db_id'], ex['question'], ex['struct_in'], ex['seq_out']

('car_1',
 'Find the model of the car whose weight is below the average weight.',
 '| car_1 | continents : contid , continent | countries : countryid , countryname , continent | car_makers : id , maker , fullname , country | model_list : modelid , maker , model | car_names : makeid , model , make | cars_data : id , mpg , cylinders , edispl , horsepower , weight , accelerate , year',
 'select t1.model from car_names as t1 join cars_data as t2 on t1.makeid = t2.id where t2.weight < (select avg(weight) from cars_data)')

In [ ]:
r = ctu.trace_with_repatch_uskg_multi_token(
    mt_uskg.model,
    inp=inp,
    states_to_patch=[(4, ctu.layername_uskg(mt_uskg.model, 'decoder', 3))],
    states_to_unpatch=[(4, ctu.layername_uskg(mt_uskg.model, 'decoder', 4, 'cross_attn'))],
    states_to_patch_1st_pass=[(4, ctu.layername_uskg(mt_uskg.model, 'decoder', l)) for l in range(mt_uskg.num_enc_layers)],
    answers_t=pred[0],
    tokens_to_mix=(10, 20, 30, 40),
    tokens_to_mix_1st_pass=(5, 15, 25),
    tokens_to_mix_individual_indices=True,
    replace=True,
)

In [511]:
r

[tensor(0.5877, device='cuda:0'),
 tensor(1., device='cuda:0'),
 tensor(0.9988, device='cuda:0'),
 tensor(1.0000, device='cuda:0')]

In [ ]:
# # inp = ctu.make_inputs_t5(mt_uskg.tokenizer, [text_in] * 11, [dec_prompt] * 11)
# inp = ctu.make_inputs_t5(mt_uskg.tokenizer, [enc_sentence] * 11, [dec_prompt] * 11, answer=expect, device='cpu')
# # answer_t, base_score = [d[0] for d in ctu.predict_from_input_uskg(mt_uskg.model, inp)]
# # base_score = base_score.item()
# # [answer] = ctu.decode_tokens(mt_uskg.tokenizer, [answer_t])
# answer_len = 1
# if expect is not None:
#     answer_len = len(mt_uskg.tokenizer.tokenize(expect))
# with torch.no_grad():
#     answers_t, base_score = [d[0] for d in ctu.predict_from_input_uskg_multi_token(mt_uskg.model, inp, pred_len=answer_len)]
# # base_score = base_score.min().item()
# # [answer] = decode_tokens(mt.tokenizer, [answer_t])
# answer = ctu.decode_sentences(mt_uskg.tokenizer, answers_t)

# expect, answers_t, answer, base_score

In [ ]:
# # e_range = (129, 132)
# # e_range = [8, 13, 131]
# e_range = list(range(7, 14))

# r = ctu.trace_with_patch_uskg(
#     mt_uskg.model,
#     inp=inp,
#     states_to_patch=[], 
#     answers_t=answer_t, 
#     tokens_to_mix=e_range,
#     tokens_to_mix_individual_indices=True,
#     replace=True,
# )

# r

### Moving models

In [82]:
mt_uskg.model.device

device(type='cpu')

### tokenizer

In [55]:
tokenizer = mt_uskg.tokenizer

In [56]:
s = "mary: has a little  lambbb"
s_ = "mary: has a little lambbb"
tokenizer.tokenize(s), tokenizer.tokenize(s_)

(['▁mar', 'y', ':', '▁has', '▁', 'a', '▁little', '▁lamb', 'b', 'b'],
 ['▁mar', 'y', ':', '▁has', '▁', 'a', '▁little', '▁lamb', 'b', 'b'])

In [428]:
t = tokenizer(s)
t

{'input_ids': [3157, 63, 10, 65, 3, 9, 385, 17871, 115, 115, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [429]:
list(enumerate(t.tokens()))

[(0, '▁mar'),
 (1, 'y'),
 (2, ':'),
 (3, '▁has'),
 (4, '▁'),
 (5, 'a'),
 (6, '▁little'),
 (7, '▁lamb'),
 (8, 'b'),
 (9, 'b'),
 (10, '</s>')]

In [430]:
t.word_to_tokens(4)

TokenSpan(start=7, end=10)

In [431]:
ctu.decode_sentences(tokenizer, t['input_ids'][7:10])

'lambbb'

In [ ]:
"(a(a)a)".rindex(")"), "(a(a)a)".index(")")

In [ ]:
s = """| concert_singer | singer : singer_id , name ( First Last ) , country ( France , Germany , United States ) , \
song_name , song_release_year , age , is_male"""

tokenizer.tokenize(s)

In [269]:
tokenizer.decode([10])

':'

### Test struct_in parsing

In [66]:
_struct_in = """| concert_singer | stadium : stadium_id , location , name , capacity , highest , lowest , \
average | singer : singer_id , name ( First Last ) , country ( France , Germany , United States ) , \
song_name , song_release_year , age , is_male | concert : concert_id , concert_name , theme , \
stadium_id , year ( 2008 , 2012 , 2022 ) | singer_in_concert : concert_id , singer_id"""

In [71]:
ctu.parse_struct_in(_struct_in)

[((3, 'stadium'),
  [[(5, 'stadium_id'), []],
   [(7, 'location'), []],
   [(9, 'name'), []],
   [(11, 'capacity'), []],
   [(13, 'highest'), []],
   [(15, 'lowest'), []],
   [(17, 'average'), []]]),
 ((19, 'singer'),
  [[(21, 'singer_id'), []],
   [(23, 'name'), [(25, 'First Last')]],
   [(29, 'country'), [(31, 'France'), (33, 'Germany'), (35, 'United States')]],
   [(39, 'song_name'), []],
   [(41, 'song_release_year'), []],
   [(43, 'age'), []],
   [(45, 'is_male'), []]]),
 ((47, 'concert'),
  [[(49, 'concert_id'), []],
   [(51, 'concert_name'), []],
   [(53, 'theme'), []],
   [(55, 'stadium_id'), []],
   [(57, 'year'), [(59, '2008'), (61, '2012'), (63, '2022')]]]),
 ((66, 'singer_in_concert'),
  [[(68, 'concert_id'), []], [(70, 'singer_id'), []]])]

In [84]:
_text_in = text_in

enc_sentence = f"{_text_in}; structed knowledge: {_struct_in}"
enc_tokenized = mt_uskg.tokenizer(enc_sentence)

In [85]:
token_ranges_dict = ctu.find_struct_name_ranges(mt_uskg.tokenizer, enc_tokenized['input_ids'], _struct_in)

In [ ]:
token_ranges_dict

In [ ]:
for d_key, d in token_ranges_dict.items():
    for name, ranges in d.items():
        for s, e in ranges:
            recs_name = ctu.decode_sentences(mt_uskg.tokenizer, enc_tokenized['input_ids'][s:e])
            print(f'{d_key}\t{name}\t{recs_name}')

### Stanza

In [99]:
import stanza

In [ ]:
nlp = stanza.Pipeline(lang='en', processors='tokenize,mwt,pos,lemma')

In [102]:
doc = nlp('cars data')
print(*[f'word: {word.text+" "}\tlemma: {word.lemma}' for sent in doc.sentences for word in sent.words], sep='\n')

word: cars 	lemma: car
word: data 	lemma: datum


In [122]:
doc = nlp('cars data')
doc.sentences[0].words[0]

{
  "id": 1,
  "text": "cars",
  "lemma": "car",
  "upos": "NOUN",
  "xpos": "NNS",
  "feats": "Number=Plur",
  "start_char": 0,
  "end_char": 4
}

### Tokenizer-fast / uskg

In [331]:
ex = processed_spider_dev[416]

In [332]:
# TODO: add special token (<, <=) to t5-tokenizer 
# could use uskg tokenizer, but it's not TokenizerFast 

text_in = ex['text_in']
struct_in = ex['struct_in']

txt = f"{text_in}; structed knowledge: {struct_in}"

tokenized_txt = mt_uskg.tokenizer([txt], max_length=1024, padding="max_length", truncation=True)

device = mt_uskg.model.device
_output = mt_uskg.model.generate(
    torch.tensor(tokenized_txt.data['input_ids'], dtype=int, device=device),
    torch.tensor(tokenized_txt.data['attention_mask'], dtype=int, device=device),
    num_beams=1, 
    max_length=256
)

_output

tensor([[    0,  1738,     3,     9,   208,   122,   599,  5525,   834,   858,
           834, 26416,    61,    45,  7071,   213,   539,   834,  1201, 32100,
          2464,     1]], device='cuda:0')

In [334]:
mt_uskg.tokenizer.batch_decode(
    _output,
    skip_special_tokens=True 
)

['select avg(num_of_staff) from museum where open_year 2009']

In [333]:
mt_uskg.tokenizer_uskg.batch_decode(
    _output,
    skip_special_tokens=True 
)

['select avg(num_of_staff) from museum where open_year  < 2009']

In [48]:
mt_uskg.tokenizer.tokenize('t3.')

['▁', 't', '3.']

### Prepare input for chatgpt

In [401]:
ex = processed_spider_dev[503]

In [402]:
text_in = ex['text_in']
struct_in = ex['struct_in']

enc_sentence = f"{text_in}; structed knowledge: {struct_in}"

print(f"{enc_sentence} => {ex['seq_out']}")

How many battles did not lose any ship with tonnage '225'?; structed knowledge: | battle_death | battle : id , name , date , bulgarian_commander , latin_commander , result | ship : lost_in_battle , id , name , tonnage , ship_type , location , disposition_of_ship | death : caused_by_ship_id , id , note , killed , injured => select count(*) from battle where id not in ( select lost_in_battle from ship where tonnage = '225' );


### One-time patches

In [173]:
# # For exp2: ['trace_results'][.]: ['%col%_score'] -> ['%node%_score']

# res_dir = '/home/yshao/Projects/rome/results/exp2_text_struct_interaction/'
# in_path = os.path.join(res_dir, 'exp=2_train_column-tmp.jsonl')
# out_path = os.path.join(res_dir, 'exp=2_train_column.jsonl')

# with open(in_path, 'r') as f:
#     all_results = [json.loads(l) for l in f]

# for ex_d in all_results:
#     for d in ex_d['trace_results']:
#         if not d['is_good_sample']:
#             continue
#         d['r_node_score'] = d['r_col_score']
#         d['r_struct_no_node_score'] = d['r_struct_no_col_score']
#         d['r_node_corrupt_all_score'] = d['r_col_corrupt_all_score']
#         del d['r_col_score']
#         del d['r_struct_no_col_score']
#         del d['r_col_corrupt_all_score']

# all_results.sort(key=lambda d: d['ex_id'])
        
# with open(out_path, 'w') as f:
#     for d in all_results:
#         f.write(json.dumps(d, indent=None) + '\n')
        

In [ ]:
d

### RE test

In [197]:
dec_target = ' a b a c aa da a '
subject = 'bb'
m = re.search(fr'\W({subject})\W', dec_target)
m

In [198]:
m = re.finditer(fr'\W({subject})\W', dec_target)
m

In [199]:
m = list(m)
m

[]

### exp4 plotting

In [455]:
def plot_uskg_enc_attention(d, savepdf=None):
    ## Assume 16 heads, 24 layers (T5 large config)
    
    ## encoder self attention 
    inspect_layers = [0, 6, 12, 18, 23]
    att_dict = d['attentions']
    
    cand_len = len(att_dict['enc_cand_tokens'])
    head_len = len(att_dict['enc_head_tokens'])

    fig_w = 22
    fig_h = (0.11*cand_len + 1) * head_len
    fig, ax_list = plt.subplots(
        nrows=head_len,
        ncols=len(inspect_layers),
        squeeze=False,
        figsize=(fig_w, fig_h))

    att_mat = ctu.nested_list_processing(att_dict['enc_attn'], func=float)
    att_mat = np.array(att_mat)
    
    for expect_i in range(len(att_dict['enc_head_tokens'])):
        for l_id, layer in enumerate(inspect_layers):
            val_mat = att_mat[layer, :, expect_i, :]  # layer, all heads, expect tok i -> all toks 
            val_mat = val_mat.transpose()    # (cand_toks, n_heads)
            x_labels = range(val_mat.shape[1])
            y_labels = att_dict['enc_cand_tokens']
            title_toks = att_dict['enc_head_tokens'][:expect_i] + [f"*{att_dict['enc_head_tokens'][expect_i]}*"]
            title = f"L{layer}  Head token: {' '.join(title_toks)}\n"
            
            ax = ax_list[expect_i, l_id]
            _draw_single_plot_2(ax,
                                val_mat=val_mat, 
                                x_labels=x_labels, 
                                y_labels=y_labels,
                                title=title)
            
    fig.tight_layout()
    if savepdf:
        plt.savefig(savepdf, bbox_inches="tight")
        plt.close()
    else:
        plt.show()

In [471]:
def plot_uskg_cross_attention(d, savepdf=None):
    ## Assume 16 heads, 24 layers (T5 large config)
    
    ## decoder cross attention 
    inspect_layers = [0, 6, 12, 18, 23]
    att_dict = d['attentions']
    cand_len = len(att_dict['cross_cand_tokens'])
    head_len = len(att_dict['cross_head_tokens'])
    prompt_len = len(att_dict['dec_cand_tokens'])

    fig_w = 22
    fig_h = (0.11*cand_len + 1) * head_len
    fig, ax_list = plt.subplots(
        nrows=head_len,
        ncols=len(inspect_layers),
        squeeze=False,
        figsize=(fig_w, fig_h))

    att_mat = ctu.nested_list_processing(att_dict['cross_attn'], func=float)
    att_mat = np.array(att_mat)
    
    for expect_i in range(head_len):
        for l_id, layer in enumerate(inspect_layers):
            val_mat = att_mat[layer, :, expect_i, :]  # layer, all heads, expect tok i -> all toks 
            val_mat = val_mat.transpose()    # (cand_toks, n_heads)
            x_labels = range(val_mat.shape[1])
            y_labels = att_dict['cross_cand_tokens']
            # a small hack to use gold tokens from dec_prompt (dec_cand_tokens) for previous steps and predicted tokens at this step 
            # dec_prompt ends with the first (head_len-1) tokens of the target node 
            title_toks = att_dict['dec_cand_tokens'][prompt_len - (head_len-1) : prompt_len - (head_len-1) + expect_i] + [f"*{att_dict['cross_head_tokens'][expect_i]}*"]
            title = f"L{layer}  Head token: {' '.join(title_toks)}\n"
            
            ax = ax_list[expect_i, l_id]
            _draw_single_plot_2(ax,
                                val_mat=val_mat, 
                                x_labels=x_labels, 
                                y_labels=y_labels,
                                title=title)
            
    fig.tight_layout()
    if savepdf:
        plt.savefig(savepdf, bbox_inches="tight")
        plt.close()
    else:
        plt.show()

In [472]:
# d['attentions']['dec_cand_tokens'][-5:]

In [473]:
savepdf_path = '/home/yshao/Projects/rome/results/figs/exp4_inspect_attention/tmp-6-cross.pdf'
plot_uskg_cross_attention(d, savepdf=savepdf_path)

In [474]:
def plot_uskg_dec_attention(d, savepdf=None):
    ## Assume 16 heads, 24 layers (T5 large config)
    
    ## decoder self attention 
    inspect_layers = [0, 6, 12, 18, 23]
    att_dict = d['attentions']
    prompt_len = cand_len = len(att_dict['dec_cand_tokens'])
    head_len = len(att_dict['dec_head_tokens'])

    fig_w = 22
    fig_h = (0.11*cand_len + 1) * head_len
    fig, ax_list = plt.subplots(
        nrows=head_len,
        ncols=len(inspect_layers),
        squeeze=False,
        figsize=(fig_w, fig_h))

    att_mat = ctu.nested_list_processing(att_dict['dec_attn'], func=float)
    att_mat = np.array(att_mat)
    
    for expect_i in range(head_len):
        for l_id, layer in enumerate(inspect_layers):
            val_mat = att_mat[layer, :, expect_i, :]  # layer, all heads, expect tok i -> all toks 
            val_mat = val_mat.transpose()    # (cand_toks, n_heads)
            x_labels = range(val_mat.shape[1])
            y_labels = att_dict['dec_cand_tokens']
            # a small hack to use gold tokens from dec_prompt (dec_cand_tokens) for previous steps and predicted tokens at this step 
            # dec_prompt ends with the first (head_len-1) tokens of the target node 
            title_toks = att_dict['dec_cand_tokens'][prompt_len - (head_len-1) : prompt_len - (head_len-1) + expect_i] + [f"*{att_dict['dec_head_tokens'][expect_i]}*"]
            title = f"L{layer}  Head token: {' '.join(title_toks)}\n"
            
            ax = ax_list[expect_i, l_id]
            _draw_single_plot_2(ax,
                                val_mat=val_mat, 
                                x_labels=x_labels, 
                                y_labels=y_labels,
                                title=title)
            
    fig.tight_layout()
    if savepdf:
        plt.savefig(savepdf, bbox_inches="tight")
        plt.close()
    else:
        plt.show()

In [475]:
savepdf_path = '/home/yshao/Projects/rome/results/figs/exp4_inspect_attention/tmp-6-dec.pdf'
plot_uskg_dec_attention(d, savepdf=savepdf_path)

In [366]:
np.array(d['attentions']['enc_attn']).shape

(24, 16, 1, 141)

In [418]:
_att_mat = ctu.nested_list_processing(d['attentions']['enc_attn'], float)
_att_mat = np.array(_att_mat)[-1, :, 0, :]  # layer 0, all heads, expect tok 0 -> all toks 

_att_mat.shape

(16, 141)

In [ ]:
cand_len = len(d['attentions']['enc_tgt_tokens'])
fig_h = cand_len / 10

fig = plt.figure(figsize=(4, fig_h))
ax = fig.gca()

val_mat = _att_mat.transpose()
x_labels = range(val_mat.shape[1])
y_labels = d['attentions']['enc_tgt_tokens']
title_toks = d['attentions']['enc_src_tokens'][:-1] + [f"*{d['attentions']['enc_src_tokens'][-1]}*"]
title = ' '.join(title_toks)

_draw_single_plot_2(ax,
                    val_mat=val_mat, 
                    x_labels=x_labels, 
                    y_labels=y_labels,
                    title=title,
                   )
fig.tight_layout()
plt.savefig('/home/yshao/Projects/rome/results/figs/exp4_inspect_attention/tmp-1.pdf',
            bbox_inches="tight")

In [ ]:
fig, ax_list = plt.subplots(nrows=4, ncols=2)
ax_list.shape

In [431]:
# plt.close('all')

### exp5.2 attention trace

In [268]:
t = torch.randn(2,3,4,2)

In [270]:
_m = torch.zeros_like(t).bool()

In [273]:
_m[:, 1:, None, :] = True
_m

tensor([[[[False, False],
          [False, False],
          [False, False],
          [False, False]],

         [[ True,  True],
          [ True,  True],
          [ True,  True],
          [ True,  True]],

         [[ True,  True],
          [ True,  True],
          [ True,  True],
          [ True,  True]]],


        [[[False, False],
          [False, False],
          [False, False],
          [False, False]],

         [[ True,  True],
          [ True,  True],
          [ True,  True],
          [ True,  True]],

         [[ True,  True],
          [ True,  True],
          [ True,  True],
          [ True,  True]]]])

In [274]:
## K/V: (bs, seq_len, m_dim) --[k/v]--> (bs, seq_len, n_head * dim) --[shape]--> (bs, n_head, seq_len, dim)
## can corrupt v output on seq_len dimensions with certain seq_indices, giving attention ignoring these indices 
## can then restore the attention final output for other_seq_indices; no

replace=True  # True to replace with instead of add noise
noise_fn = lambda x: 0 * x

patch_spec = []
unpatch_spec = []
corrupt_spec = []

toks_to_mix = []

def patch_rep(x, layer):

    # if first_pass or (layer not in patch_spec and layer not in unpatch_spec):
    if (layer not in patch_spec) and (layer not in unpatch_spec) and (layer not in corrupt_spec):
        return x

    h = untuple(x)
    if layer in corrupt_spec:
        toks_to_mix = corrupt_spec[layer]
        if toks_to_mix:
            mix_len = len(toks_to_mix)

            noise_data = noise_fn(
                torch.from_numpy(prng(h.shape[0] - 1, mix_len, h.shape[2]))
            ).to(device=h.device, dtype=h.dtype)

            if replace:
                h[1:, toks_to_mix] = noise_data
            else:
                h[1:, toks_to_mix] += noise_data

    # If this layer is in the patch_spec, restore the uncorrupted hidden state
    # for selected tokens.
    toks_to_patch = patch_spec.get(layer, [])
    toks_to_unpatch = unpatch_spec.get(layer, [])
    # if toks_to_patch:
    #     print(f'* 2nd pass, layer: {layer}, restoring: {toks_to_patch}')
    # if toks_to_unpatch:
    #     print(f'* 2nd pass, layer: {layer}, unpatching: {toks_to_unpatch}')

    for t in toks_to_patch:
        h[1:, t] = h[0, t]
    for t in toks_to_unpatch:
        NotImplemented
        # h[1:, t] = untuple(first_pass_trace[layer].output)[1:, t]
    return x


### other temp

## (placeholder)